In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [197]:
import warnings
import gc
import os
import random
import sys
import zipfile

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier

from sklearn.model_selection import train_test_split
from datetime import datetime
from glob import glob
from tqdm import tqdm
from IPython.display import Image

# 추후에 경고 값들을 출력하지 않도록 'ignore'상태로 만들어 준다.
# 불필요한 경고창이 더 이상 출력 되지 않게 됩니다.
warnings.filterwarnings(action = 'ignore')

In [198]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [199]:
train_2017=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/train_2017_fin_0124.csv')
test_2017=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/test_2017_fin_0124.csv')
train_2018=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/train_2018_fin_0124.csv')
test_2018=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/test_2018_fin_0124.csv')
train_2019=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/train_2019_fin_0124.csv')
test_2019=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/test_2019_fin_0124.csv')
train_2020=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/train_2020_fin_0124.csv')
test_2020=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_24/test_2020_fin_0124.csv')

In [200]:
# 학과
trdf17=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/train_2017_학과수정.csv')
tsdf17=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/test_2017_학과수정.csv')
train_2017['학과']=trdf17['bq38_1']
test_2017['학과']=tsdf17['bq38_1']

trdf18=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/train_2018_학과수정.csv')
tsdf18=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/test_2018_학과수정.csv')
train_2018['학과']=trdf18['bq37_1']
test_2018['학과']=tsdf18['bq37_1']

trdf19=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/train_2019_학과수정.csv')
tsdf19=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/test_2019_학과수정.csv')
train_2019['학과']=trdf19['bq27_1']
test_2019['학과']=tsdf19['bq27_1']

trdf20=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/train_2020_학과수정.csv')
tsdf20=pd.read_csv('/content/drive/MyDrive/조달청 공모전/01_26/test_2020_학과수정.csv')
train_2020['학과']=trdf20['bq26_1']
test_2020['학과']=tsdf20['bq26_1']

In [201]:
# df1=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred17_2.csv')
# df2=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred18_2.csv')
# df3=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred19_2.csv')
# df4=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred20_2.csv')
# df=pd.concat([df1,df2,df3,df4], axis=0);df.columns

In [ ]:
# # df=pd.concat([df1,df2,df3,df4], axis=0)
# submission = pd.read_csv('C:\\Users\\User\\Downloads\\sample_submission.csv') # sample submission 불러오기
# submission['knowcode'] = df['0'].values

# submission.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\submission0126_cb.csv',index=False)

In [ ]:
# train_2017=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\train_2017_fin_0124.csv')
# test_2017=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\test_2017_fin_0124.csv')
# train_2018=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\train_2018_fin_0124.csv')
# test_2018=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\test_2018_fin_0124.csv')
# train_2019=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\train_2019_fin_0124.csv')
# test_2019=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\test_2019_fin_0124.csv')
# train_2020=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\train_2020_fin_0124.csv')
# test_2020=pd.read_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\test_2020_fin_0124.csv')

In [202]:
type_bq1 = [1,2,3,4,5,6,7,8]

train_2017['업종']='type1'
train_2017.loc[train_2017['bq1'].isin(type_bq1),'업종']='type2'
test_2017['업종']='type1'
test_2017.loc[test_2017['bq1'].isin(type_bq1),'업종']='type2'

train_2018['업종']='type1'
train_2018.loc[train_2018['bq1'].isin(type_bq1),'업종']='type2'
test_2018['업종']='type1'
test_2018.loc[test_2018['bq1'].isin(type_bq1),'업종']='type2'

train_2019['업종']='type1'
train_2019.loc[train_2019['bq1'].isin(type_bq1),'업종']='type2'
test_2019['업종']='type1'
test_2019.loc[test_2019['bq1'].isin(type_bq1),'업종']='type2'

train_2020['업종']='type1'
train_2020.loc[train_2020['bq1'].isin(type_bq1),'업종']='type2'
test_2020['업종']='type1'
test_2020.loc[test_2020['bq1'].isin(type_bq1),'업종']='type2'

In [203]:
# 범주형
train_2017[['bq1', 'bq2', 'bq39_2']] = train_2017[['bq1', 'bq2', 'bq39_2']].astype('object')
train_2018[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 'bq38_1', 
      'bq38_2']] = train_2018[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 
                         'bq38_1', 'bq38_2']].astype('object')
train_2019[['bq1', 'bq2', 'bq28_1', 'bq28_2']] = train_2019[['bq1', 'bq2', 'bq28_1', 'bq28_2']].astype('object')
train_2020[['bq1', 'bq2', 'bq27_2']] = train_2020[['bq1', 'bq2', 'bq27_2']].astype('object')

# 범주형
test_2017[['bq1', 'bq2', 'bq39_2']] = test_2017[['bq1', 'bq2', 'bq39_2']].astype('object')
test_2018[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 'bq38_1', 
      'bq38_2']] = test_2018[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 
                         'bq38_1', 'bq38_2']].astype('object')
test_2019[['bq1', 'bq2', 'bq28_1', 'bq28_2']] = test_2019[['bq1', 'bq2', 'bq28_1', 'bq28_2']].astype(int)
test_2019[['bq1', 'bq2', 'bq28_1', 'bq28_2']] = test_2019[['bq1', 'bq2', 'bq28_1', 'bq28_2']].astype('object')
test_2020[['bq1', 'bq2', 'bq27_2']] = test_2020[['bq1', 'bq2', 'bq27_2']].astype('object')

In [204]:
test_2018[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 'bq38_1', 
      'bq38_2']].head()

,bq1,bq2,bq13,bq15,bq17,bq18,bq26_1a,bq26_2a,bq26_3a,bq26_4a,bq38_1,bq38_2
0,1,4,3,4,3,3,0,0,0,0,0,6
1,10,3,1,1,1,1,2,2,2,0,1,0
2,20,4,3,4,3,3,2,2,2,1,0,6
3,20,4,3,4,3,3,2,2,2,1,0,6
4,19,1,2,1,1,3,0,0,0,0,4,0


In [205]:
train_2017=train_2017.drop(['bq1','bq1_1','bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30',
       'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1','bq38_2'],axis=1)
test_2017=test_2017.drop(['bq1','bq1_1','bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30',
       'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1','bq38_2'],axis=1)

In [206]:
train_2018=train_2018.drop(['bq1','bq1_1','bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2','bq28_1',
       'bq29', 'bq30', 'bq31', 'bq32', 'bq33', 'bq37_1'],axis=1)
test_2018=test_2018.drop(['bq1','bq1_1','bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2','bq28_1',
       'bq29', 'bq30', 'bq31', 'bq32', 'bq33', 'bq37_1'],axis=1)

In [207]:
train_2019=train_2019.drop(['bq1','bq1_1'],axis=1)
test_2019=test_2019.drop(['bq1','bq1_1'],axis=1)

In [208]:
train_2020=train_2020.drop(['bq1','bq1_new'],axis=1)
test_2020=test_2020.drop(['bq1','bq1_new'],axis=1)

In [209]:
train_2017.describe(include='object')

,bq2,bq39_2,학과,업종
count,9486,9486,9486,9486
unique,6,7,104,2
top,1,1,자연공,type1
freq,6133,6913,1228,5941


In [210]:
cat17=['bq2', 'bq39_2', '학과','업종', 'bq41_1_cut', 'bq41_2_cut',
       'bq41_3_cut', 'bq37_cut']
cat18=['bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a',
       'bq26_3a', 'bq26_4a', 'bq38_1', 'bq38_2', '학과','업종', 'cq50',
       'bq41_1_cut', 'bq41_2_cut', 'bq41_3_cut', 'bq36_cut']
cat19=['bq2', 'bq28_1', 'bq28_2', '학과','업종', 'bq30_cut', 'bq26_cut',
       'bq31_1_cut', 'bq31_2_cut', 'bq31_3_cut']
cat20=['bq2', 'bq27_2', '학과','업종', 'bq30_1_cut', 'bq30_2_cut',
       'bq30_3_cut', 'bq25_cut', 'bq29_cut']

In [211]:
train_2017[['bq41_1_cut', 'bq41_2_cut', 'bq41_3_cut', 'bq37_cut']]=train_2017[['bq41_1_cut', 'bq41_2_cut', 'bq41_3_cut', 'bq37_cut']].astype('object')
test_2017[['bq41_1_cut', 'bq41_2_cut', 'bq41_3_cut', 'bq37_cut']]=test_2017[['bq41_1_cut', 'bq41_2_cut', 'bq41_3_cut', 'bq37_cut']].astype('object')
train_2017=train_2017.drop(['bq39_1'],axis=1)
test_2017=test_2017.drop(['bq39_1'],axis=1)

In [212]:
train_2018[['bq41_1_cut','bq41_2_cut', 'bq41_3_cut','bq36_cut','학과']]=train_2018[['bq41_1_cut','bq41_2_cut', 'bq41_3_cut','bq36_cut','학과']].astype('object')
test_2018[['bq41_1_cut','bq41_2_cut', 'bq41_3_cut','bq36_cut','학과']]=test_2018[['bq41_1_cut','bq41_2_cut', 'bq41_3_cut','bq36_cut','학과']].astype('object')

In [213]:
train_2019[['학과', 'bq30_cut', 'bq26_cut', 'bq31_1_cut', 'bq31_2_cut', 'bq31_3_cut']]=train_2019[['학과', 'bq30_cut', 'bq26_cut', 'bq31_1_cut', 'bq31_2_cut', 'bq31_3_cut']].astype('object')
test_2019[['학과', 'bq30_cut', 'bq26_cut', 'bq31_1_cut', 'bq31_2_cut', 'bq31_3_cut']]=test_2019[['학과', 'bq30_cut', 'bq26_cut', 'bq31_1_cut', 'bq31_2_cut', 'bq31_3_cut']].astype('object')

In [214]:
train_2020[['학과','bq30_1_cut', 'bq30_2_cut', 'bq30_3_cut','bq25_cut', 'bq29_cut']]=train_2020[['학과','bq30_1_cut', 'bq30_2_cut', 'bq30_3_cut','bq25_cut', 'bq29_cut']].astype('object')
test_2020[['학과','bq30_1_cut', 'bq30_2_cut', 'bq30_3_cut','bq25_cut', 'bq29_cut']]=test_2020[['학과','bq30_1_cut', 'bq30_2_cut', 'bq30_3_cut','bq25_cut', 'bq29_cut']].astype('object')

In [ ]:
train_2018.describe().columns.values

In [215]:
train_2017=train_2017.drop(['Unnamed: 0','idx','aq1', 'aq2',
       'aq3', 'aq4', 'aq5', 'aq6', 'aq7', 'aq8', 'aq9', 'aq10', 'aq11',
       'aq12', 'aq13', 'aq14', 'aq15', 'aq16', 'aq17', 'aq18', 'aq19',
       'aq20', 'aq21', 'aq22', 'aq23', 'aq24', 'aq25', 'aq26', 'aq27',
       'aq28', 'aq29', 'aq30', 'aq31', 'aq32', 'aq33', 'aq34', 'aq35',
       'aq36', 'aq37', 'aq38', 'aq39', 'aq40', 'aq41'],axis=1)
test_2017=test_2017.drop(['Unnamed: 0','idx','aq1', 'aq2',
       'aq3', 'aq4', 'aq5', 'aq6', 'aq7', 'aq8', 'aq9', 'aq10', 'aq11',
       'aq12', 'aq13', 'aq14', 'aq15', 'aq16', 'aq17', 'aq18', 'aq19',
       'aq20', 'aq21', 'aq22', 'aq23', 'aq24', 'aq25', 'aq26', 'aq27',
       'aq28', 'aq29', 'aq30', 'aq31', 'aq32', 'aq33', 'aq34', 'aq35',
       'aq36', 'aq37', 'aq38', 'aq39', 'aq40', 'aq41'],axis=1)
train_2018=train_2018.drop(['Unnamed: 0','idx','cq50'],axis=1)
test_2018=test_2018.drop(['Unnamed: 0','idx','cq50'],axis=1)
train_2019=train_2019.drop(['Unnamed: 0','kq1',
       'kq2', 'kq3', 'kq4', 'kq5', 'kq6', 'kq7', 'kq8', 'kq9', 'kq10',
       'kq11', 'kq12', 'kq13', 'kq14', 'kq15', 'kq16', 'kq17', 'kq18',
       'kq19', 'kq20', 'kq21', 'kq22', 'kq23', 'kq24', 'kq25', 'kq26',
       'kq27', 'kq28', 'kq29', 'kq30', 'kq31', 'kq32', 'kq33'],axis=1)
test_2019=test_2019.drop(['Unnamed: 0','kq1',
       'kq2', 'kq3', 'kq4', 'kq5', 'kq6', 'kq7', 'kq8', 'kq9', 'kq10',
       'kq11', 'kq12', 'kq13', 'kq14', 'kq15', 'kq16', 'kq17', 'kq18',
       'kq19', 'kq20', 'kq21', 'kq22', 'kq23', 'kq24', 'kq25', 'kq26',
       'kq27', 'kq28', 'kq29', 'kq30', 'kq31', 'kq32', 'kq33'],axis=1)
train_2020=train_2020.drop(['saq1', 'saq2', 'saq3',
       'saq4', 'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10', 'saq11',
       'saq12', 'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18',
       'saq19', 'saq20', 'saq21', 'saq22', 'saq23', 'saq24', 'saq25',
       'saq26', 'saq27', 'saq28', 'saq29', 'saq30', 'saq31', 'saq32',
       'saq33', 'saq34', 'saq35', 'saq36', 'saq37', 'saq38', 'saq39',
       'saq40', 'saq41', 'saq42', 'saq43', 'saq44'],axis=1)
test_2020=test_2020.drop(['saq1', 'saq2', 'saq3',
       'saq4', 'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10', 'saq11',
       'saq12', 'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18',
       'saq19', 'saq20', 'saq21', 'saq22', 'saq23', 'saq24', 'saq25',
       'saq26', 'saq27', 'saq28', 'saq29', 'saq30', 'saq31', 'saq32',
       'saq33', 'saq34', 'saq35', 'saq36', 'saq37', 'saq38', 'saq39',
       'saq40', 'saq41', 'saq42', 'saq43', 'saq44'],axis=1)

In [216]:
train_2018[['cq1', 'cq2', 'cq3', 'cq4', 'cq5', 'cq6', 'cq7', 'cq8', 'cq9',
       'cq10', 'cq11', 'cq12', 'cq13', 'cq14', 'cq15', 'cq16', 'cq17',
       'cq18', 'cq19', 'cq20', 'cq21', 'cq22', 'cq23', 'cq24', 'cq25',
       'cq26', 'cq27', 'cq28', 'cq29', 'cq30', 'cq31', 'cq32', 'cq33',
       'cq34', 'cq35', 'cq36', 'cq37', 'cq38', 'cq39', 'cq40', 'cq41',
       'cq42', 'cq43', 'cq44', 'cq45', 'cq46', 'cq47', 'cq48', 'cq49',
       'cq50_1', 'cq50_2', 'cq50_3', 'cq50_4', 'cq50_5', 'cq50_6',
       'cq50_7', 'cq50_8', 'iq1', 'iq2', 'iq3', 'iq4', 'iq5', 'iq6',
       'bq3', 'bq4', 'bq5', 'bq5_1', 'bq6', 'bq7', 'bq8_1', 'bq8_2',
       'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1', 'bq12_2', 'bq12_3',
       'bq12_4', 'bq12_5', 'bq14', 'bq16', 'bq19', 'bq20', 'bq21',
       'bq221', 'bq222', 'bq223', 'bq231', 'bq232', 'bq233', 'bq234',
       'bq235', 'bq241', 'bq242', 'bq243', 'bq244', 'bq245', 'bq25',
       'bq25_1', 'bq26_1', 'bq26_2', 'bq26_3', 'bq26_4', 'bq27', 'bq28',
       'bq34', 'bq35', 'bq36', 'bq37', 'bq38', 'bq39', 'bq40', 'bq41_1',
       'bq41_2', 'bq41_3', 'bq4_1', 'bq8_sum', 'bq12_sum']]=train_2018[['cq1', 'cq2', 'cq3', 'cq4', 'cq5', 'cq6', 'cq7', 'cq8', 'cq9',
       'cq10', 'cq11', 'cq12', 'cq13', 'cq14', 'cq15', 'cq16', 'cq17',
       'cq18', 'cq19', 'cq20', 'cq21', 'cq22', 'cq23', 'cq24', 'cq25',
       'cq26', 'cq27', 'cq28', 'cq29', 'cq30', 'cq31', 'cq32', 'cq33',
       'cq34', 'cq35', 'cq36', 'cq37', 'cq38', 'cq39', 'cq40', 'cq41',
       'cq42', 'cq43', 'cq44', 'cq45', 'cq46', 'cq47', 'cq48', 'cq49',
       'cq50_1', 'cq50_2', 'cq50_3', 'cq50_4', 'cq50_5', 'cq50_6',
       'cq50_7', 'cq50_8', 'iq1', 'iq2', 'iq3', 'iq4', 'iq5', 'iq6',
       'bq3', 'bq4', 'bq5', 'bq5_1', 'bq6', 'bq7', 'bq8_1', 'bq8_2',
       'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1', 'bq12_2', 'bq12_3',
       'bq12_4', 'bq12_5', 'bq14', 'bq16', 'bq19', 'bq20', 'bq21',
       'bq221', 'bq222', 'bq223', 'bq231', 'bq232', 'bq233', 'bq234',
       'bq235', 'bq241', 'bq242', 'bq243', 'bq244', 'bq245', 'bq25',
       'bq25_1', 'bq26_1', 'bq26_2', 'bq26_3', 'bq26_4', 'bq27', 'bq28',
       'bq34', 'bq35', 'bq36', 'bq37', 'bq38', 'bq39', 'bq40', 'bq41_1',
       'bq41_2', 'bq41_3', 'bq4_1', 'bq8_sum', 'bq12_sum']].astype(int)

In [217]:
train_2019[['sq1', 'sq2', 'sq3', 'sq4', 'sq5', 'sq6', 'sq7', 'sq8', 'sq9',
       'sq10', 'sq11', 'sq12', 'sq13', 'sq14', 'sq15', 'sq16', 'kq1_1',
       'kq1_2', 'kq2_1', 'kq2_2', 'kq3_1', 'kq3_2', 'kq4_1', 'kq4_2',
       'kq5_1', 'kq5_2', 'kq6_1', 'kq6_2', 'kq7_1', 'kq7_2', 'kq8_1',
       'kq8_2', 'kq9_1', 'kq9_2', 'kq10_1', 'kq10_2', 'kq11_1', 'kq11_2',
       'kq12_1', 'kq12_2', 'kq13_1', 'kq13_2', 'kq14_1', 'kq14_2',
       'kq15_1', 'kq15_2', 'kq16_1', 'kq16_2', 'kq17_1', 'kq17_2',
       'kq18_1', 'kq18_2', 'kq19_1', 'kq19_2', 'kq20_1', 'kq20_2',
       'kq21_1', 'kq21_2', 'kq22_1', 'kq22_2', 'kq23_1', 'kq23_2',
       'kq24_1', 'kq24_2', 'kq25_1', 'kq25_2', 'kq26_1', 'kq26_2',
       'kq27_1', 'kq27_2', 'kq28_1', 'kq28_2', 'kq29_1', 'kq29_2',
       'kq30_1', 'kq30_2', 'kq31_1', 'kq31_2', 'kq32_1', 'kq32_2',
       'kq33_1', 'kq33_2', 'bq3', 'bq4', 'bq5', 'bq5_1', 'bq6', 'bq7',
       'bq8_1', 'bq8_2', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1',
       'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5', 'bq13_1', 'bq13_2',
       'bq13_3', 'bq14_1', 'bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq15',
       'bq16_1', 'bq16_2', 'bq16_3', 'bq16_4', 'bq16_5', 'bq17', 'bq18_1',
       'bq18_2', 'bq18_3', 'bq18_4', 'bq18_5', 'bq18_6', 'bq18_7',
       'bq18_8', 'bq18_9', 'bq19', 'bq20', 'bq21_1', 'bq21_2', 'bq21_3',
       'bq25', 'bq26', 'bq27', 'bq28', 'bq29', 'bq30', 'bq31_1', 'bq31_2',
       'bq31_3', 'bq8_sum', 'bq12_sum', 'bq13_sum',
       'bq14_sum', 'bq16_sum', 'bq21_sum', 'bq4_1']]=train_2019[['sq1', 'sq2', 'sq3', 'sq4', 'sq5', 'sq6', 'sq7', 'sq8', 'sq9',
       'sq10', 'sq11', 'sq12', 'sq13', 'sq14', 'sq15', 'sq16', 'kq1_1',
       'kq1_2', 'kq2_1', 'kq2_2', 'kq3_1', 'kq3_2', 'kq4_1', 'kq4_2',
       'kq5_1', 'kq5_2', 'kq6_1', 'kq6_2', 'kq7_1', 'kq7_2', 'kq8_1',
       'kq8_2', 'kq9_1', 'kq9_2', 'kq10_1', 'kq10_2', 'kq11_1', 'kq11_2',
       'kq12_1', 'kq12_2', 'kq13_1', 'kq13_2', 'kq14_1', 'kq14_2',
       'kq15_1', 'kq15_2', 'kq16_1', 'kq16_2', 'kq17_1', 'kq17_2',
       'kq18_1', 'kq18_2', 'kq19_1', 'kq19_2', 'kq20_1', 'kq20_2',
       'kq21_1', 'kq21_2', 'kq22_1', 'kq22_2', 'kq23_1', 'kq23_2',
       'kq24_1', 'kq24_2', 'kq25_1', 'kq25_2', 'kq26_1', 'kq26_2',
       'kq27_1', 'kq27_2', 'kq28_1', 'kq28_2', 'kq29_1', 'kq29_2',
       'kq30_1', 'kq30_2', 'kq31_1', 'kq31_2', 'kq32_1', 'kq32_2',
       'kq33_1', 'kq33_2', 'bq3', 'bq4', 'bq5', 'bq5_1', 'bq6', 'bq7',
       'bq8_1', 'bq8_2', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1',
       'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5', 'bq13_1', 'bq13_2',
       'bq13_3', 'bq14_1', 'bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq15',
       'bq16_1', 'bq16_2', 'bq16_3', 'bq16_4', 'bq16_5', 'bq17', 'bq18_1',
       'bq18_2', 'bq18_3', 'bq18_4', 'bq18_5', 'bq18_6', 'bq18_7',
       'bq18_8', 'bq18_9', 'bq19', 'bq20', 'bq21_1', 'bq21_2', 'bq21_3',
       'bq25', 'bq26', 'bq27', 'bq28', 'bq29', 'bq30', 'bq31_1', 'bq31_2',
       'bq31_3', 'bq8_sum', 'bq12_sum', 'bq13_sum',
       'bq14_sum', 'bq16_sum', 'bq21_sum', 'bq4_1']].astype(int)

In [218]:
test_2020[['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2',
       'saq4_1', 'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2',
       'saq7_1', 'saq7_2', 'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2',
       'saq10_1', 'saq10_2', 'saq11_1', 'saq11_2', 'saq12_1', 'saq12_2',
       'saq13_1', 'saq13_2', 'saq14_1', 'saq14_2', 'saq15_1', 'saq15_2',
       'saq16_1', 'saq16_2', 'saq17_1', 'saq17_2', 'saq18_1', 'saq18_2',
       'saq19_1', 'saq19_2', 'saq20_1', 'saq20_2', 'saq21_1', 'saq21_2',
       'saq22_1', 'saq22_2', 'saq23_1', 'saq23_2', 'saq24_1', 'saq24_2',
       'saq25_1', 'saq25_2', 'saq26_1', 'saq26_2', 'saq27_1', 'saq27_2',
       'saq28_1', 'saq28_2', 'saq29_1', 'saq29_2', 'saq30_1', 'saq30_2',
       'saq31_1', 'saq31_2', 'saq32_1', 'saq32_2', 'saq33_1', 'saq33_2',
       'saq34_1', 'saq34_2', 'saq35_1', 'saq35_2', 'saq36_1', 'saq36_2',
       'saq37_1', 'saq37_2', 'saq38_1', 'saq38_2', 'saq39_1', 'saq39_2',
       'saq40_1', 'saq40_2', 'saq41_1', 'saq41_2', 'saq42_1', 'saq42_2',
       'saq43_1', 'saq43_2', 'saq44_1', 'saq44_2', 'vq1', 'vq2', 'vq3',
       'vq4', 'vq5', 'vq6', 'vq7', 'vq8', 'vq9', 'vq10', 'vq11', 'vq12',
       'vq13', 'bq3', 'bq4', 'bq5', 'bq5_1', 'bq6', 'bq7', 'bq8_1',
       'bq8_2', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1', 'bq12_2',
       'bq12_3', 'bq12_4', 'bq12_5', 'bq13_1', 'bq13_2', 'bq13_3',
       'bq14_1', 'bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6',
       'bq14_7', 'bq15', 'bq16_1', 'bq16_2', 'bq16_3', 'bq16_4', 'bq16_5',
       'bq16_6', 'bq16_7', 'bq16_8', 'bq16_9', 'bq16_10', 'bq17',
       'bq18_1', 'bq18_2', 'bq18_3', 'bq18_4', 'bq18_5', 'bq18_6',
       'bq18_7', 'bq18_8', 'bq18_9', 'bq19', 'bq20', 'bq21_1', 'bq21_2',
       'bq21_3', 'bq21_4', 'bq22_1', 'bq22_2', 'bq22_3', 'bq22_4',
       'bq22_5', 'bq22_6', 'bq23_1', 'bq23_2', 'bq23_3', 'bq24', 'bq25',
       'bq26', 'bq27_1', 'bq28', 'bq29', 'bq30_1', 'bq30_2', 'bq30_3',
       'lic_freq', 'bq8_sum', 'bq12_sum', 'bq13_sum',
       'bq14_sum']]=test_2020[['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2',
       'saq4_1', 'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2',
       'saq7_1', 'saq7_2', 'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2',
       'saq10_1', 'saq10_2', 'saq11_1', 'saq11_2', 'saq12_1', 'saq12_2',
       'saq13_1', 'saq13_2', 'saq14_1', 'saq14_2', 'saq15_1', 'saq15_2',
       'saq16_1', 'saq16_2', 'saq17_1', 'saq17_2', 'saq18_1', 'saq18_2',
       'saq19_1', 'saq19_2', 'saq20_1', 'saq20_2', 'saq21_1', 'saq21_2',
       'saq22_1', 'saq22_2', 'saq23_1', 'saq23_2', 'saq24_1', 'saq24_2',
       'saq25_1', 'saq25_2', 'saq26_1', 'saq26_2', 'saq27_1', 'saq27_2',
       'saq28_1', 'saq28_2', 'saq29_1', 'saq29_2', 'saq30_1', 'saq30_2',
       'saq31_1', 'saq31_2', 'saq32_1', 'saq32_2', 'saq33_1', 'saq33_2',
       'saq34_1', 'saq34_2', 'saq35_1', 'saq35_2', 'saq36_1', 'saq36_2',
       'saq37_1', 'saq37_2', 'saq38_1', 'saq38_2', 'saq39_1', 'saq39_2',
       'saq40_1', 'saq40_2', 'saq41_1', 'saq41_2', 'saq42_1', 'saq42_2',
       'saq43_1', 'saq43_2', 'saq44_1', 'saq44_2', 'vq1', 'vq2', 'vq3',
       'vq4', 'vq5', 'vq6', 'vq7', 'vq8', 'vq9', 'vq10', 'vq11', 'vq12',
       'vq13', 'bq3', 'bq4', 'bq5', 'bq5_1', 'bq6', 'bq7', 'bq8_1',
       'bq8_2', 'bq8_3', 'bq9', 'bq10', 'bq11', 'bq12_1', 'bq12_2',
       'bq12_3', 'bq12_4', 'bq12_5', 'bq13_1', 'bq13_2', 'bq13_3',
       'bq14_1', 'bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6',
       'bq14_7', 'bq15', 'bq16_1', 'bq16_2', 'bq16_3', 'bq16_4', 'bq16_5',
       'bq16_6', 'bq16_7', 'bq16_8', 'bq16_9', 'bq16_10', 'bq17',
       'bq18_1', 'bq18_2', 'bq18_3', 'bq18_4', 'bq18_5', 'bq18_6',
       'bq18_7', 'bq18_8', 'bq18_9', 'bq19', 'bq20', 'bq21_1', 'bq21_2',
       'bq21_3', 'bq21_4', 'bq22_1', 'bq22_2', 'bq22_3', 'bq22_4',
       'bq22_5', 'bq22_6', 'bq23_1', 'bq23_2', 'bq23_3', 'bq24', 'bq25',
       'bq26', 'bq27_1', 'bq28', 'bq29', 'bq30_1', 'bq30_2', 'bq30_3',
       'lic_freq', 'bq8_sum', 'bq12_sum', 'bq13_sum',
       'bq14_sum']].astype(int)

# bq1 인덱스로 나누기

In [225]:
train_2017['idx']=train_2017.index
test_2017['idx']=test_2017.index

train_2018['idx']=train_2018.index
test_2018['idx']=test_2018.index

train_2019['idx']=train_2019.index
test_2019['idx']=test_2019.index

train_2020['idx']=train_2020.index
test_2020['idx']=test_2020.index

In [245]:
idx_tr_17_1 = train_2017[train_2017['업종'] == 'type1']['idx'].to_list()
idx_tr_17_2 = train_2017[train_2017['업종'] == 'type2']['idx'].to_list()
idx_tr_18_1 = train_2018[train_2018['업종'] == 'type1']['idx'].to_list()
idx_tr_18_2 = train_2018[train_2018['업종'] == 'type2']['idx'].to_list()
idx_tr_19_1 = train_2019[train_2019['업종'] == 'type1']['idx'].to_list()
idx_tr_19_2 = train_2019[train_2019['업종'] == 'type2']['idx'].to_list()
idx_tr_20_1 = train_2020[train_2020['업종'] == 'type1']['idx'].to_list()
idx_tr_20_2 = train_2020[train_2020['업종'] == 'type2']['idx'].to_list()

idx_ts_17_1 = test_2017[test_2017['업종'] == 'type1']['idx'].to_list()
idx_ts_17_2 = test_2017[test_2017['업종'] == 'type2']['idx'].to_list()
idx_ts_18_1 = test_2018[test_2018['업종'] == 'type1']['idx'].to_list()
idx_ts_18_2 = test_2018[test_2018['업종'] == 'type2']['idx'].to_list()
idx_ts_19_1 = test_2019[test_2019['업종'] == 'type1']['idx'].to_list()
idx_ts_19_2 = test_2019[test_2019['업종'] == 'type2']['idx'].to_list()
idx_ts_20_1 = test_2020[test_2020['업종'] == 'type1']['idx'].to_list()
idx_ts_20_2 = test_2020[test_2020['업종'] == 'type2']['idx'].to_list()

In [254]:
train_2018['bq40']=train_2018['bq40'].astype(float)
test_2018['bq40']=test_2018['bq40'].astype(float)

y17=train_2017['knowcode']; X17=train_2017.drop('knowcode', axis=1, inplace=False)
y18=train_2018['knowcode']; X18=train_2018.drop('knowcode', axis=1, inplace=False)
y19=train_2019['knowcode']; X19=train_2019.drop('knowcode', axis=1, inplace=False)
y20=train_2020['knowcode']; X20=train_2020.drop('knowcode', axis=1, inplace=False)
X17=pd.get_dummies(X17)
X19=pd.get_dummies(X19)
X20=pd.get_dummies(X20)
X18=pd.get_dummies(X18)
test18=pd.get_dummies(test_2018)
test17=pd.get_dummies(test_2017)
test19=pd.get_dummies(test_2019)
test20=pd.get_dummies(test_2020)
test18=test18.drop(['bq41_3_cut_1'],axis=1)

In [255]:
cat17=['bq2', 'bq39_2', '학과','업종', 'bq41_1_cut', 'bq41_2_cut',
       'bq41_3_cut', 'bq37_cut']
cat18=['bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a',
       'bq26_3a', 'bq26_4a', 'bq38_1', 'bq38_2', '학과','업종', 'cq50',
       'bq41_1_cut', 'bq41_2_cut', 'bq41_3_cut', 'bq36_cut']
cat19=['bq2', 'bq28_1', 'bq28_2', '학과','업종', 'bq30_cut', 'bq26_cut',
       'bq31_1_cut', 'bq31_2_cut', 'bq31_3_cut']
cat20=['bq2', 'bq27_2', '학과','업종', 'bq30_1_cut', 'bq30_2_cut',
       'bq30_3_cut', 'bq25_cut', 'bq29_cut']

In [256]:
cat17_1=['bq2_1', 'bq2_2', 'bq2_3', 'bq2_4', 'bq2_5', 'bq2_6', 'bq39_2_1',
       'bq39_2_2', 'bq39_2_3', 'bq39_2_4', 'bq39_2_5', 'bq39_2_6',
       'bq39_2_7', '학과_간호', '학과_건축', '학과_게임', '학과_경영', '학과_경제', '학과_경찰',
       '학과_공업', '학과_공업계', '학과_공학', '학과_광고', '학과_교통', '학과_국사', '학과_국어',
       '학과_군사', '학과_금속', '학과_금형', '학과_금형설계', '학과_기계', '학과_기관', '학과_기타',
       '학과_기타언어', '학과_도시', '학과_독어', '학과_동물', '학과_디자인', '학과_레크레이션',
       '학과_모델', '학과_무역', '학과_무용', '학과_문과', '학과_문예창작', '학과_문화', '학과_물리',
       '학과_미디어', '학과_미술', '학과_미용', '학과_방사선', '학과_번역', '학과_법', '학과_보건',
       '학과_복지', '학과_불어', '학과_비서', '학과_사서', '학과_사진', '학과_사회', '학과_산업공',
       '학과_생명', '학과_선박', '학과_설비', '학과_섬유', '학과_세무회계', '학과_소방', '학과_수학',
       '학과_수학/통계', '학과_시스템', '학과_식품', '학과_심리', '학과_아동가족', '학과_안경',
       '학과_약학', '학과_없음', '학과_에너지', '학과_연극', '학과_영어', '학과_용접', '학과_원자력공',
       '학과_유통', '학과_음악', '학과_응급구조', '학과_의료', '학과_이과', '학과_이벤트', '학과_일어',
       '학과_자동차', '학과_자연공', '학과_자원', '학과_재료', '학과_전기전자', '학과_전산', '학과_정보',
       '학과_정치외교', '학과_제어계측공', '학과_조경', '학과_종교', '학과_중어', '학과_지리', '학과_철도',
       '학과_철학', '학과_체육/스포츠', '학과_축산', '학과_치기공', '학과_치료', '학과_치위생',
       '학과_컴퓨터공학', '학과_토목', '학과_항공', '학과_항해', '학과_해양', '학과_행정', '학과_호텔관광',
       '학과_화학', '학과_환경', '학과_회화', '업종_type1', '업종_type2', 'bq41_1_cut_0',
       'bq41_1_cut_1', 'bq41_1_cut_2', 'bq41_1_cut_3', 'bq41_1_cut_4',
       'bq41_1_cut_5', 'bq41_1_cut_6', 'bq41_1_cut_7', 'bq41_1_cut_8',
       'bq41_1_cut_9', 'bq41_2_cut_0', 'bq41_2_cut_1', 'bq41_2_cut_2',
       'bq41_2_cut_3', 'bq41_2_cut_4', 'bq41_2_cut_5', 'bq41_2_cut_6',
       'bq41_2_cut_7', 'bq41_2_cut_8', 'bq41_2_cut_9', 'bq41_3_cut_0',
       'bq41_3_cut_1', 'bq41_3_cut_2', 'bq41_3_cut_3', 'bq41_3_cut_4',
       'bq41_3_cut_5', 'bq41_3_cut_6', 'bq41_3_cut_7', 'bq41_3_cut_8',
       'bq41_3_cut_9', 'bq37_cut_0', 'bq37_cut_1', 'bq37_cut_2',
       'bq37_cut_3', 'bq37_cut_4', 'bq37_cut_5', 'bq37_cut_6',
       'bq37_cut_7']

cat19_1=['bq2_1', 'bq2_2', 'bq2_3', 'bq2_4', 'bq2_5', 'bq2_6', 'bq28_1_0',
       'bq28_1_1', 'bq28_1_2', 'bq28_1_3', 'bq28_1_4', 'bq28_2_0',
       'bq28_2_5', 'bq28_2_6', 'bq28_2_7', '학과_MBA', '학과_간호', '학과_건축',
       '학과_게임', '학과_경영', '학과_경제', '학과_경찰', '학과_고졸', '학과_공업', '학과_공학',
       '학과_광고', '학과_교통', '학과_국사', '학과_국어', '학과_군사', '학과_금속', '학과_금형설계',
       '학과_기계', '학과_기관', '학과_기록관리', '학과_기타', '학과_기타언어', '학과_도시', '학과_독어',
       '학과_동물', '학과_디자인', '학과_마술학', '학과_모델', '학과_무역', '학과_무용', '학과_문과',
       '학과_문예창작', '학과_문화', '학과_물리', '학과_미디어', '학과_미술', '학과_미용', '학과_방사선',
       '학과_방사선학', '학과_법', '학과_보건', '학과_복지', '학과_불어', '학과_비서', '학과_사무자동화',
       '학과_사서', '학과_사진', '학과_사회', '학과_산업공', '학과_산업공학', '학과_생명', '학과_선박',
       '학과_설비', '학과_섬유', '학과_세무회계', '학과_소방', '학과_수학', '학과_수학/통계',
       '학과_시스템', '학과_식품', '학과_심리', '학과_아동가족', '학과_안경', '학과_안전공', '학과_약학',
       '학과_어업', '학과_없음', '학과_에너지', '학과_연극', '학과_영어', '학과_용접', '학과_원자력 공',
       '학과_원자력공', '학과_음악', '학과_응급구조', '학과_의료', '학과_이과', '학과_일어', '학과_자동차',
       '학과_자연공', '학과_자원', '학과_재료', '학과_전기전자', '학과_전산', '학과_정보', '학과_정치외교',
       '학과_제어계측공', '학과_조경', '학과_종교', '학과_중어', '학과_철학', '학과_체육/스포츠',
       '학과_축산', '학과_치기공', '학과_치기공학', '학과_치료', '학과_치위생', '학과_컴퓨터공학',
       '학과_토목', '학과_토목공학', '학과_항공', '학과_항해', '학과_해양', '학과_행정', '학과_호텔관광',
       '학과_화학', '학과_환경', '업종_type1', '업종_type2', 'bq30_cut_0',
       'bq30_cut_1', 'bq30_cut_2', 'bq30_cut_3', 'bq30_cut_4',
       'bq30_cut_5', 'bq30_cut_6', 'bq30_cut_7', 'bq30_cut_8',
       'bq26_cut_0', 'bq26_cut_1', 'bq26_cut_2', 'bq26_cut_3',
       'bq26_cut_4', 'bq26_cut_5', 'bq26_cut_6', 'bq26_cut_7',
       'bq31_1_cut_0', 'bq31_1_cut_1', 'bq31_1_cut_2', 'bq31_1_cut_3',
       'bq31_1_cut_4', 'bq31_1_cut_5', 'bq31_1_cut_6', 'bq31_1_cut_7',
       'bq31_1_cut_8', 'bq31_1_cut_9', 'bq31_2_cut_0', 'bq31_2_cut_1',
       'bq31_2_cut_2', 'bq31_2_cut_3', 'bq31_2_cut_4', 'bq31_2_cut_5',
       'bq31_2_cut_6', 'bq31_2_cut_7', 'bq31_2_cut_8', 'bq31_2_cut_9',
       'bq31_3_cut_0', 'bq31_3_cut_1', 'bq31_3_cut_2', 'bq31_3_cut_3',
       'bq31_3_cut_4', 'bq31_3_cut_5', 'bq31_3_cut_6', 'bq31_3_cut_7',
       'bq31_3_cut_8', 'bq31_3_cut_9']
cat20_1=['bq2_1', 'bq2_2', 'bq2_3', 'bq2_4', 'bq2_5', 'bq27_2_1',
       'bq27_2_2', 'bq27_2_3', 'bq27_2_4', 'bq27_2_5', 'bq27_2_6',
       'bq27_2_7', '학과_간호', '학과_건축', '학과_경영', '학과_경제', '학과_경찰', '학과_공업',
       '학과_공학', '학과_광고', '학과_교통', '학과_국사', '학과_국어', '학과_그래픽', '학과_금속',
       '학과_금형설계', '학과_기계', '학과_기관', '학과_기타', '학과_기타언어', '학과_도시', '학과_동물',
       '학과_디자인', '학과_모델', '학과_무역', '학과_무용', '학과_문과', '학과_문예창작', '학과_문화',
       '학과_물리', '학과_미디어', '학과_미술', '학과_미용', '학과_방사선', '학과_법', '학과_보건',
       '학과_복지', '학과_비서', '학과_사서', '학과_사진', '학과_사회', '학과_산림', '학과_산업공',
       '학과_생명', '학과_선박', '학과_설비', '학과_섬유', '학과_세무회계', '학과_소방', '학과_수학/통계',
       '학과_스튜어디스', '학과_시스템', '학과_식품', '학과_심리', '학과_아동가족', '학과_안경',
       '학과_안전공', '학과_약학', '학과_없음', '학과_에너지', '학과_연극', '학과_영어', '학과_용접',
       '학과_원자력공', '학과_유통', '학과_음악', '학과_음향', '학과_응급구조', '학과_의료', '학과_이과',
       '학과_일어', '학과_자동차', '학과_자연공', '학과_자원', '학과_장례지도', '학과_재료',
       '학과_전기전자', '학과_전산', '학과_정보', '학과_정치외교', '학과_제어계측', '학과_제어계측공',
       '학과_조경', '학과_조선공', '학과_종교', '학과_중어', '학과_지구', '학과_지리', '학과_지적',
       '학과_천문', '학과_철도', '학과_철학', '학과_체육/스포츠', '학과_축산', '학과_치기공', '학과_치료',
       '학과_치위생', '학과_컴퓨터공학', '학과_토목', '학과_항공', '학과_항해', '학과_해양', '학과_행정',
       '학과_호텔관광', '학과_화학', '학과_환경', '학과_회화', '업종_type1', '업종_type2',
       'bq30_1_cut_0', 'bq30_1_cut_1', 'bq30_1_cut_2', 'bq30_1_cut_3',
       'bq30_1_cut_4', 'bq30_1_cut_5', 'bq30_1_cut_6', 'bq30_1_cut_7',
       'bq30_1_cut_8', 'bq30_1_cut_9', 'bq30_2_cut_0', 'bq30_2_cut_1',
       'bq30_2_cut_2', 'bq30_2_cut_3', 'bq30_2_cut_4', 'bq30_2_cut_5',
       'bq30_2_cut_6', 'bq30_2_cut_7', 'bq30_2_cut_8', 'bq30_2_cut_9',
       'bq30_3_cut_0', 'bq30_3_cut_1', 'bq30_3_cut_2', 'bq30_3_cut_3',
       'bq30_3_cut_4', 'bq30_3_cut_5', 'bq30_3_cut_6', 'bq30_3_cut_7',
       'bq30_3_cut_8', 'bq30_3_cut_9', 'bq25_cut_1', 'bq25_cut_2',
       'bq25_cut_3', 'bq25_cut_4', 'bq25_cut_5', 'bq25_cut_6',
       'bq25_cut_7', 'bq29_cut_0', 'bq29_cut_1', 'bq29_cut_2',
       'bq29_cut_3', 'bq29_cut_4', 'bq29_cut_5', 'bq29_cut_6',
       'bq29_cut_7']

In [257]:
cat18_1=['bq2_1', 'bq2_2', 'bq2_3', 'bq2_4', 'bq2_5', 'bq2_6',
       'bq13_1', 'bq13_2', 'bq13_3', 'bq15_1', 'bq15_2', 'bq15_3',
       'bq15_4', 'bq17_1', 'bq17_2', 'bq17_3', 'bq18_1', 'bq18_2',
       'bq18_3', 'bq26_1a_0', 'bq26_1a_1', 'bq26_1a_2', 'bq26_1a_3',
       'bq26_2a_0', 'bq26_2a_1', 'bq26_2a_2', 'bq26_2a_3', 'bq26_3a_0',
       'bq26_3a_1', 'bq26_3a_2', 'bq26_3a_3', 'bq26_4a_0', 'bq26_4a_1',
       'bq26_4a_2', 'bq26_4a_3', 'bq38_1_0', 'bq38_1_1', 'bq38_1_2',
       'bq38_1_3', 'bq38_1_4', 'bq38_2_0', 'bq38_2_5', 'bq38_2_6',
       'bq38_2_7', '학과_MBA', '학과_간호', '학과_건축', '학과_게임', '학과_경영', '학과_경제',
       '학과_경찰', '학과_고졸', '학과_공업', '학과_공학', '학과_광고', '학과_교통', '학과_국사',
       '학과_국어', '학과_군사', '학과_금속', '학과_금형', '학과_기계', '학과_기관', '학과_기타',
       '학과_기타언어', '학과_도시', '학과_독어', '학과_동물', '학과_디자인', '학과_모델', '학과_무역',
       '학과_무용', '학과_문과', '학과_문예창작', '학과_문화', '학과_물리', '학과_미디어', '학과_미술',
       '학과_미용', '학과_방사선', '학과_번역', '학과_법', '학과_보건', '학과_복지', '학과_불어',
       '학과_비서', '학과_사서', '학과_사진', '학과_사회', '학과_산림공', '학과_산업공', '학과_생명',
       '학과_선박', '학과_설비', '학과_섬유', '학과_세무회계', '학과_소방', '학과_수학/통계',
       '학과_시스템', '학과_식물', '학과_식품', '학과_심리', '학과_아동가족', '학과_안경', '학과_약학',
       '학과_어업', '학과_없음', '학과_에너지', '학과_연극', '학과_영어', '학과_원자력공', '학과_음악',
       '학과_응급구조', '학과_의료', '학과_의류', '학과_이과', '학과_일어', '학과_임', '학과_자동차',
       '학과_자연공', '학과_자원', '학과_재료', '학과_전기전자', '학과_전산', '학과_정보', '학과_정치외교',
       '학과_조경', '학과_종교', '학과_중어', '학과_지리', '학과_철학', '학과_청각', '학과_체육/스포츠',
       '학과_축산', '학과_치기공', '학과_치료', '학과_치위생', '학과_카지노', '학과_컴퓨터공학',
       '학과_토목', '학과_토목공학', '학과_항공', '학과_항해', '학과_해양', '학과_행정', '학과_호텔관광',
       '학과_화학', '학과_환경', '업종_type1', '업종_type2', 'bq41_1_cut_0',
       'bq41_1_cut_1', 'bq41_1_cut_2', 'bq41_1_cut_3', 'bq41_1_cut_4',
       'bq41_1_cut_5', 'bq41_1_cut_6', 'bq41_1_cut_7', 'bq41_1_cut_8',
       'bq41_1_cut_9', 'bq41_2_cut_0', 'bq41_2_cut_1', 'bq41_2_cut_2',
       'bq41_2_cut_3', 'bq41_2_cut_4', 'bq41_2_cut_5', 'bq41_2_cut_6',
       'bq41_2_cut_7', 'bq41_2_cut_8', 'bq41_2_cut_9', 'bq41_3_cut_0',
       'bq41_3_cut_2', 'bq41_3_cut_3', 'bq41_3_cut_4', 'bq41_3_cut_5',
       'bq41_3_cut_6', 'bq41_3_cut_7', 'bq41_3_cut_8', 'bq41_3_cut_9',
       'bq36_cut_0', 'bq36_cut_1', 'bq36_cut_2', 'bq36_cut_3',
       'bq36_cut_4', 'bq36_cut_5', 'bq36_cut_6', 'bq36_cut_7']

In [241]:
X17.shape

(9486, 302)

In [258]:
type1_X17=X17.iloc[idx_tr_17_1 , :]
type1_y17=y17.iloc[idx_tr_17_1]
type1_test17=test17.iloc[idx_ts_17_1 , :]

type1_X18=X18.iloc[idx_tr_18_1 , :]
type1_y18=y18.iloc[idx_tr_18_1]
type1_test18=test18.iloc[idx_ts_18_1 , :]

type1_X19=X19.iloc[idx_tr_19_1 , :]
type1_y19=y19.iloc[idx_tr_19_1]
type1_test19=test19.iloc[idx_ts_19_1 , :]

type1_X20=X20.iloc[idx_tr_20_1 , :]
type1_y20=y20.iloc[idx_tr_20_1]
type1_test20=test20.iloc[idx_ts_20_1 , :]

# 업종 Type 1

In [248]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_state=42)

In [ ]:
model17 = CatBoostClassifier(random_state=42)
model17.fit(type1_X17, type1_y17,cat_features=cat17_1)

In [ ]:
pred17 = model17.predict(type1_test17)
preddf=pd.DataFrame(pred17)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred17_type1.csv')


In [260]:
model18 = CatBoostClassifier(random_state=42)
model18.fit(type1_X18, type1_y18,cat_features=cat18_1)


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.086451
0:	learn: 6.0079362	total: 9.18s	remaining: 2h 32m 47s
1:	learn: 5.9419480	total: 17.7s	remaining: 2h 27m 14s


KeyboardInterrupt: ignored

In [ ]:
pred18 = model18.predict(type1_test18)
preddf=pd.DataFrame(pred18)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred18_type1.csv')

In [ ]:
model.fit(type1_X19, type1_y19,cat_features=cat19_1)


In [ ]:
pred19 = model.predict(type1_test19)
preddf=pd.DataFrame(pred19)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred19_type1.csv')

In [ ]:
model.fit(type1_X20, type1_y20,cat_features=cat20_1)

In [ ]:
pred20 = model.predict(type1_test20)
preddf=pd.DataFrame(pred20)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred20_type2.csv')

# 업종 Type 2

In [264]:
type2_X17=X17.iloc[idx_tr_17_2 , :]
type2_y17=y17.iloc[idx_tr_17_2]
type2_test17=test17.iloc[idx_ts_17_2 , :]

type2_X18=X18.iloc[idx_tr_18_2 , :]
type2_y18=y18.iloc[idx_tr_18_2]
type2_test18=test18.iloc[idx_ts_18_2 , :]

type2_X19=X19.iloc[idx_tr_19_2 , :]
type2_y19=y19.iloc[idx_tr_19_2]
type2_test19=test19.iloc[idx_ts_19_2 , :]

type2_X20=X20.iloc[idx_tr_20_2 , :]
type2_y20=y20.iloc[idx_tr_20_2]
type2_test20=test20.iloc[idx_ts_20_2 , :]

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_state=42)

In [ ]:
model17 = CatBoostClassifier(random_state=42)
model17.fit(type2_X17, type2_y17,cat_features=cat17_1)

In [ ]:
pred17 = model17.predict(type2_test17)
preddf=pd.DataFrame(pred17)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred17_type2.csv')


In [ ]:
model18 = CatBoostClassifier(random_state=42)
model18.fit(type2_X18, type2_y18,cat_features=cat18_1)


In [ ]:
pred18 = model18.predict(type2_test18)
preddf=pd.DataFrame(pred18)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred18_type2.csv')

In [ ]:
model.fit(type2_X19, type2_y19,cat_features=cat19_1)


In [ ]:
pred19 = model.predict(type2_test19)
preddf=pd.DataFrame(pred19)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred19_type2.csv')

In [ ]:
model.fit(type2_X20, type2_y20,cat_features=cat20_1)

In [ ]:
pred20 = model.predict(type2_test20)
preddf=pd.DataFrame(pred20)
preddf.to_csv('C:\\Users\\User\\Downloads\\01_24-20220124T144539Z-001\\01_24\\pred20_type2.csv')

# 모든 변수

* 2017

In [ ]:
tr17_all=train_2017.drop(['Unnamed: 0', 'idx', 'bq19_1' ,'bq30' ,'bq31' ,'bq32' ,'bq33' ,'bq34' ,'bq38_1','bq4_1a',
       'bq4_1b', 'bq4_1c', 'bq5_2'], axis=1)
ts17_all=test_2017.drop(['Unnamed: 0', 'idx','bq19_1' ,'bq30' ,'bq31' ,'bq32' ,'bq33' ,'bq34' ,'bq38_1','bq4_1a',
       'bq4_1b', 'bq4_1c', 'bq5_2'],axis=1)
df=pd.get_dummies(tr17_all[['bq1_1','bq38_2']])
tr17_all=pd.concat([tr17_all,df],axis=1)
df=pd.get_dummies(ts17_all[['bq1_1','bq38_2']])
ts17_all=pd.concat([ts17_all,df],axis=1)
# train_2017[['bq1_1', 'bq4_1', 'bq38_2']] # 업종/자격증개수/5개과
tr17_all=tr17_all.drop(['bq1_1','bq38_2'],axis=1)
ts17_all=ts17_all.drop(['bq1_1','bq38_2'],axis=1)


* 2018

In [ ]:
tr18_all=train_2018.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1',
       'idx','bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1', 'bq29', 'bq30', 'bq31', 'bq32','bq33', 'bq37_1'], axis=1)
ts18_all=test_2018.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'Unnamed: 0.1.1.1.1.1.1.1', 'idx','bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1', 'bq29', 'bq30', 'bq31', 'bq32','bq33', 'bq37_1'], axis=1)
# train_2018[['bq1_1', 'bq4_1', '학과']] 
df=pd.get_dummies(tr18_all[['bq1_1','학과']])
tr18_all=pd.concat([tr18_all,df],axis=1)
df=pd.get_dummies(ts18_all[['bq1_1','학과']])
ts18_all=pd.concat([ts18_all,df],axis=1)
tr18_all=tr18_all.drop(['bq1_1','학과'],axis=1)
ts18_all=ts18_all.drop(['bq1_1','학과'],axis=1)

* 2019

In [ ]:
tr19_all=train_2019.drop(['Unnamed: 0','Unnamed: 0.1', 'idx','bq4_1a', 'bq4_1b', 'bq4_1c','bq5_2', 'bq18_10','bq20_1', 
                      'bq22', 'bq23', 'bq24', 'bq27_1'],axis=1)
ts19_all=test_2019.drop(['Unnamed: 0','Unnamed: 0.1', 'idx', 'bq4_1a', 'bq4_1b', 'bq4_1c','bq5_2', 'bq18_10','bq20_1',
                     'bq22', 'bq23', 'bq24', 'bq27_1'],axis=1)
train_2019[['bq1_1','bq4_1', '학과']]
df=pd.get_dummies(tr19_all[['bq1_1','학과']])
tr19_all=pd.concat([tr19_all,df],axis=1)
df=pd.get_dummies(ts19_all[['bq1_1','학과']])
ts19_all=pd.concat([ts19_all,df],axis=1)
tr19_all=tr19_all.drop(['bq1_1','학과'],axis=1)
ts19_all=ts19_all.drop(['bq1_1','학과'],axis=1)

* 2020

In [ ]:
tr20_all=train_2020.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'idx','bq4_1a', 'bq4_1b', 'bq4_1c','bq5_2','bq20_1','bq26_1'], axis=1)
ts20_all=test_2020.drop(['Unnamed: 0', 'Unnamed: 0.1', 'idx','bq4_1a', 'bq4_1b', 'bq4_1c','bq5_2','bq20_1','bq26_1'],axis=1)
 #'학과', 'bq1_new'
df=pd.get_dummies(tr20_all[['bq1_new','학과']])
tr20_all=pd.concat([tr20_all,df],axis=1)
df=pd.get_dummies(ts20_all[['bq1_new','학과']])
ts20_all=pd.concat([ts20_all,df],axis=1)
tr20_all=tr20_all.drop(['bq1_new','학과'],axis=1)
ts20_all=ts20_all.drop(['bq1_new','학과'],axis=1)

* 범주형 처리

In [ ]:
# # 범주형
# tr17_all[['bq1', 'bq2', 'bq39_2']] = tr17_all[['bq1', 'bq2', 'bq39_2']].astype('object')
# tr18_all[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 'bq38_1', 
#       'bq38_2']] = tr18_all[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 
#                          'bq38_1', 'bq38_2']].astype('object')
# tr19_all[['bq1', 'bq2', 'bq28_1', 'bq28_2']] = tr19_all[['bq1', 'bq2', 'bq28_1', 'bq28_2']].astype('object')
# tr20_all[['bq1', 'bq2', 'bq27_2']] = tr20_all[['bq1', 'bq2', 'bq27_2']].astype('object')

# # 범주형
# ts17_all[['bq1', 'bq2', 'bq39_2']] = ts17_all[['bq1', 'bq2', 'bq39_2']].astype('object')
# ts18_all[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 'bq38_1', 
#       'bq38_2']] = ts18_all[['bq1', 'bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 
#                          'bq38_1', 'bq38_2']].astype('object')
# ts19_all[['bq1', 'bq2', 'bq28_1', 'bq28_2']] = ts19_all[['bq1', 'bq2', 'bq28_1', 'bq28_2']].astype('object')
ts20_all[['bq1', 'bq2', 'bq27_2']] = ts20_all[['bq1', 'bq2', 'bq27_2']].astype('object')

In [ ]:
cat17=[ 'bq2', 'bq39_2','bq1_1_0', 'bq1_1_a',
       'bq1_1_b', 'bq1_1_c', 'bq1_1_d', 'bq1_1_e', 'bq1_1_f', 'bq1_1_g',
       'bq1_1_h', 'bq1_1_i', 'bq1_1_j', 'bq38_2_기타', 'bq38_2_문과', 'bq38_2_없음',
       'bq38_2_예체능', 'bq38_2_의예', 'bq38_2_이과']
cat18=['bq2', 'bq13', 'bq15', 'bq17', 'bq18', 'bq26_1a', 'bq26_2a', 'bq26_3a', 'bq26_4a', 'bq38_1', 
      'bq38_2','bq1_1_a',
       'bq1_1_b', 'bq1_1_c', 'bq1_1_e', 'bq1_1_f', 'bq1_1_g',
       'bq1_1_h', 'bq1_1_j', 'bq1_1_y', '학과_기타', '학과_문과', '학과_예체능', '학과_의료',
       '학과_이과']
cat19=['bq2', 'bq28_1', 'bq28_2','bq1_1_a', 'bq1_1_b',
       'bq1_1_c', 'bq1_1_d', 'bq1_1_e', 'bq1_1_f', 'bq1_1_g', 'bq1_1_h',
       'bq1_1_j', 'bq1_1_y', '학과_기타', '학과_문과', '학과_예체능', '학과_의료', '학과_이과']
cat20=[ 'bq2', 'bq27_2','bq1_new_a', 'bq1_new_b', 'bq1_new_c', 'bq1_new_d', 'bq1_new_e',
       'bq1_new_f', 'bq1_new_g', 'bq1_new_h', 'bq1_new_i', 'bq1_new_j',
       '학과_기타', '학과_문과', '학과_예체능', '학과_의료', '학과_이과']


In [ ]:
# 1,2 -> 1, 0 변경
for col in tr17_all.columns:
    if len(tr17_all[col].unique())==2:
        tr17_all.loc[tr17_all[col]==2,col] = 0
        
# 1,2 -> 1, 0 변경
for col in tr18_all.columns:
    if len(tr18_all[col].unique())==2:
        tr18_all.loc[tr18_all[col]==2, col] = 0
        
for col in tr19_all.columns:
    if len(tr19_all[col].unique())==2:
        tr19_all.loc[tr19_all[col]==2, col] = 0
        
for col in tr20_all.columns:
    if len(tr20_all[col].unique())==2:
        tr20_all.loc[tr20_all[col]==2, col] = 0

In [ ]:
# 0, 1, 2는 범주형 열임
for col in tr17_all.columns:
    if tr17_all[col].unique().all == [0,1,2]:
        tr17_all[col] = tr17_all[col].astype('object')
        
# 0, 1, 2는 범주형 열임
for col in tr18_all.columns:
    if tr18_all[col].unique().all == [0,1,2]:
        tr18_all[col] = tr18_all[col].astype('object')
        
# 0, 1, 2는 범주형 열임
for col in tr19_all.columns:
    if tr19_all[col].unique().all == [0,1,2]:
        tr19_all[col] = tr19_all[col].astype('object')
        
# 0, 1, 2는 범주형 열임
for col in tr20_all.columns:
    if tr20_all[col].unique().all == [0,1,2]:
        tr20_all[col] = tr20_all[col].astype('object')

In [ ]:
# 1,2 -> 1, 0 변경
for col in ts17_all.columns:
    if len(ts17_all[col].unique())==2:
        ts17_all.loc[ts17_all[col]==2,col] = 0
        
# 1,2 -> 1, 0 변경
for col in ts18_all.columns:
    if len(ts18_all[col].unique())==2:
        ts18_all.loc[ts18_all[col]==2, col] = 0
        
for col in ts19_all.columns:
    if len(ts19_all[col].unique())==2:
        ts19_all.loc[ts19_all[col]==2, col] = 0
        
for col in ts20_all.columns:
    if len(ts20_all[col].unique())==2:
        ts20_all.loc[ts20_all[col]==2, col] = 0

In [ ]:
# 0, 1, 2는 범주형 열임
for col in ts17_all.columns:
    if ts17_all[col].unique().all == [0,1,2]:
        ts17_all[col] = ts17_all[col].astype('object')
        
# 0, 1, 2는 범주형 열임
for col in ts18_all.columns:
    if ts18_all[col].unique().all == [0,1,2]:
        ts18_all[col] = ts18_all[col].astype('object')
        
# 0, 1, 2는 범주형 열임
for col in ts19_all.columns:
    if ts19_all[col].unique().all == [0,1,2]:
        ts19_all[col] = ts19_all[col].astype('object')
        
# 0, 1, 2는 범주형 열임
for col in ts20_all.columns:
    if ts20_all[col].unique().all == [0,1,2]:
        ts20_all[col] = ts20_all[col].astype('object')

# 중복변수 제거

In [ ]:
tr17_del=tr17_all.drop(['aq1_1', 'aq1_2', 'aq2_1', 'aq2_2', 'aq3_1', 'aq3_2', 'aq4_1', 'aq4_2',
       'aq5_1', 'aq5_2', 'aq6_1', 'aq6_2', 'aq7_1', 'aq7_2', 'aq8_1', 'aq8_2',
       'aq9_1', 'aq9_2', 'aq10_1', 'aq10_2', 'aq11_1', 'aq11_2', 'aq12_1',
       'aq12_2', 'aq13_1', 'aq13_2', 'aq14_1', 'aq14_2', 'aq15_1', 'aq15_2',
       'aq16_1', 'aq16_2', 'aq17_1', 'aq17_2', 'aq18_1', 'aq18_2', 'aq19_1',
       'aq19_2', 'aq20_1', 'aq20_2', 'aq21_1', 'aq21_2', 'aq22_1', 'aq22_2',
       'aq23_1', 'aq23_2', 'aq24_1', 'aq24_2', 'aq25_1', 'aq25_2', 'aq26_1',
       'aq26_2', 'aq27_1', 'aq27_2', 'aq28_1', 'aq28_2', 'aq29_1', 'aq29_2',
       'aq30_1', 'aq30_2', 'aq31_1', 'aq31_2', 'aq32_1', 'aq32_2', 'aq33_1',
       'aq33_2', 'aq34_1', 'aq34_2', 'aq35_1', 'aq35_2', 'aq36_1', 'aq36_2',
       'aq37_1', 'aq37_2', 'aq38_1', 'aq38_2', 'aq39_1', 'aq39_2', 'aq40_1',
       'aq40_2', 'aq41_1', 'aq41_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1','bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq15_1','bq15_2', 'bq15_3',
       'bq18_1', 'bq18_2', 'bq18_3', 'bq18_4', 'bq18_5', 'bq18_6', 'bq18_7',
       'bq1'],axis=1)
ts17_del=ts17_all.drop(['aq1_1', 'aq1_2', 'aq2_1', 'aq2_2', 'aq3_1', 'aq3_2', 'aq4_1', 'aq4_2',
       'aq5_1', 'aq5_2', 'aq6_1', 'aq6_2', 'aq7_1', 'aq7_2', 'aq8_1', 'aq8_2',
       'aq9_1', 'aq9_2', 'aq10_1', 'aq10_2', 'aq11_1', 'aq11_2', 'aq12_1',
       'aq12_2', 'aq13_1', 'aq13_2', 'aq14_1', 'aq14_2', 'aq15_1', 'aq15_2',
       'aq16_1', 'aq16_2', 'aq17_1', 'aq17_2', 'aq18_1', 'aq18_2', 'aq19_1',
       'aq19_2', 'aq20_1', 'aq20_2', 'aq21_1', 'aq21_2', 'aq22_1', 'aq22_2',
       'aq23_1', 'aq23_2', 'aq24_1', 'aq24_2', 'aq25_1', 'aq25_2', 'aq26_1',
       'aq26_2', 'aq27_1', 'aq27_2', 'aq28_1', 'aq28_2', 'aq29_1', 'aq29_2',
       'aq30_1', 'aq30_2', 'aq31_1', 'aq31_2', 'aq32_1', 'aq32_2', 'aq33_1',
       'aq33_2', 'aq34_1', 'aq34_2', 'aq35_1', 'aq35_2', 'aq36_1', 'aq36_2',
       'aq37_1', 'aq37_2', 'aq38_1', 'aq38_2', 'aq39_1', 'aq39_2', 'aq40_1',
       'aq40_2', 'aq41_1', 'aq41_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1','bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq15_1','bq15_2', 'bq15_3',
       'bq18_1', 'bq18_2', 'bq18_3', 'bq18_4', 'bq18_5', 'bq18_6', 'bq18_7',
       'bq1'],axis=1)

In [ ]:
tr18_del=tr18_all.drop(['cq50_1', 'cq50_2', 'cq50_3', 'cq50_4','cq50_5', 'cq50_6', 'cq50_7', 'cq50_8',
                        'bq8_1', 'bq8_2', 'bq8_3',
                        'bq12_1', 'bq12_2','bq12_3', 'bq12_4', 'bq12_5',
                        'bq1'],axis=1)
ts18_del=ts18_all.drop(['cq50_1', 'cq50_2', 'cq50_3', 'cq50_4','cq50_5', 'cq50_6', 'cq50_7', 'cq50_8',
                        'bq8_1', 'bq8_2', 'bq8_3',
                        'bq12_1', 'bq12_2','bq12_3', 'bq12_4', 'bq12_5',
                        'bq1'],axis=1)

In [ ]:
tr19_del=tr19_all.drop(['sq1', 'sq2', 'sq3', 'sq4', 'sq5', 'sq6', 'sq7', 'sq8', 'sq9', 'sq10',
       'sq11', 'sq12', 'sq13', 'sq14', 'sq15', 'sq16',
       'bq8_1', 'bq8_2', 'bq8_3','bq12_1', 'bq12_2',
       'bq12_3', 'bq12_4', 'bq12_5', 'bq13_1', 'bq13_2', 'bq13_3', 'bq14_1',
       'bq14_2', 'bq14_3', 'bq14_4', 'bq14_5','bq16_1', 'bq16_2',
       'bq16_3', 'bq16_4', 'bq16_5', 'bq21_1', 'bq21_2', 'bq21_3',
       'bq1'],axis=1)
ts19_del=ts19_all.drop(['sq1', 'sq2', 'sq3', 'sq4', 'sq5', 'sq6', 'sq7', 'sq8', 'sq9', 'sq10',
       'sq11', 'sq12', 'sq13', 'sq14', 'sq15', 'sq16',
       'bq8_1', 'bq8_2', 'bq8_3','bq12_1', 'bq12_2',
       'bq12_3', 'bq12_4', 'bq12_5', 'bq13_1', 'bq13_2', 'bq13_3', 'bq14_1',
       'bq14_2', 'bq14_3', 'bq14_4', 'bq14_5','bq16_1', 'bq16_2',
       'bq16_3', 'bq16_4', 'bq16_5', 'bq21_1', 'bq21_2', 'bq21_3',
       'bq1'],axis=1)

In [ ]:
tr20_del=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       'bq1'],axis=1)
ts20_del=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       'bq1'],axis=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, learning_curve
from sklearn.metrics import f1_score
from sklearn import preprocessing, metrics

In [ ]:
y17=tr17_all['knowcode']; X17=tr17_del.drop('knowcode', axis=1, inplace=False)
y18=tr18_all['knowcode']; X18=tr18_del.drop('knowcode', axis=1, inplace=False)
y19=tr19_all['knowcode']; X19=tr19_del.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_del.drop('knowcode', axis=1, inplace=False)

In [ ]:
tr20_all[['학과','bq1_new','bq1']].describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
col=['학과','bq1_new']
# tr20_all['학과']=encoder.fit_transform(tr20_all['학과'])
# tr20_all['bq1_new']=encoder.fit_transform(tr20_all['bq1_new'])
# ts20_all['학과']=encoder.fit_transform(ts20_all['학과'])
# ts20_all['bq1_new']=encoder.fit_transform(ts20_all['bq1_new'])
tr20_all[['bq1','학과','bq1_new']]=tr20_all[['bq1','학과','bq1_new']].astype(str)
tr20_all[['bq1','학과','bq1_new']]=ts20_all[['bq1','학과','bq1_new']].astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

In [ ]:
y20=tr20_all['knowcode']; X20=tr20_all.drop('knowcode', axis=1, inplace=False)

In [ ]:
X20.columns[100:]

In [ ]:
model1 = CatBoostClassifier( task_type = "GPU" )
model1.fit(X17, y17,cat_features=cat17, early_stopping_rounds=50)
pred17=model1.predict(ts17_del)

In [ ]:
pred17df=pd.DataFrame(pred17)
pred17df.to_csv('/content/drive/MyDrive/잡데이터제출/pred17.csv')

In [ ]:
model1 = CatBoostClassifier(task_type = "GPU"  )
model1.fit(X18, y18,cat_features=cat18,early_stopping_rounds=50)
pred18=model1.predict(ts18_del)

In [ ]:
pred17df=pd.DataFrame(pred18)
pred17df.to_csv('/content/drive/MyDrive/잡데이터제출/pred18.csv')

In [ ]:
model1 = CatBoostClassifier( task_type = "GPU" )
model1.fit(X19, y19,cat_features=cat19,early_stopping_rounds=50)
pred19=model1.predict(ts19_del)

In [ ]:
pred17df=pd.DataFrame(pred19)
pred17df.to_csv('/content/drive/MyDrive/잡데이터제출/pred19.csv')

In [ ]:
model1 = CatBoostClassifier(task_type = "GPU"  )
model1.fit(X20, y20,cat_features=cat20,early_stopping_rounds=50)
pred20=model1.predict(ts20_del)

In [ ]:
pred17df=pd.DataFrame(pred20)
pred17df.to_csv('/content/drive/MyDrive/잡데이터제출/pred20.csv')

In [ ]:
a=np.concatenate((pred17, pred18,pred19,pred20), axis=None)
submission = pd.read_csv('/content/drive/MyDrive/조달청 공모전/know_data/sample_submission.csv') # sample submission 불러오기
submission['knowcode'] = a

submission.to_csv('/content/drive/MyDrive/잡데이터제출/submission0122_del_cb.csv', index=False)

In [ ]:
aa=np.concatenate((y17, y18,y19,y20), axis=None)
f1 = f1_score(aa, a, average='macro')
print(f1)

In [ ]:
tr20_del['bq1']=tr20_del['bq1'].astype(str)
ts20_del['bq1']=ts20_del['bq1'].astype(str)

# 모델링

## 20 all 모델링 0.484

In [ ]:
y17=tr17_all['knowcode']; X17=tr17_all.drop('knowcode', axis=1, inplace=False)
y18=tr18_all['knowcode']; X18=tr18_all.drop('knowcode', axis=1, inplace=False)
y19=tr19_all['knowcode']; X19=tr19_all.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_all.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## 20 del 모델링 0.466

In [ ]:
# y17=tr17_all['knowcode']; X17=tr17_all.drop('knowcode', axis=1, inplace=False)
# y18=tr18_all['knowcode']; X18=tr18_all.drop('knowcode', axis=1, inplace=False)
# y19=tr19_all['knowcode']; X19=tr19_all.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_del.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
X20.describe(include='object')

In [ ]:
X20=X20.drop(['학과','bq1_new'],axis=1)

In [ ]:
clfs = [
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## **20 bq1알파벳(a~j) +원래범주(obj21) 0.490**

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## 20 bq1 원래범주(obj21)제거, 알파벳만 0.461

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1_new_a', 'bq1_new_b',
       'bq1_new_c', 'bq1_new_d', 'bq1_new_e', 'bq1_new_f', 'bq1_new_g',
       'bq1_new_h', 'bq1_new_i', 'bq1_new_j',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1_new_a', 'bq1_new_b',
       'bq1_new_c', 'bq1_new_d', 'bq1_new_e', 'bq1_new_f', 'bq1_new_g',
       'bq1_new_h', 'bq1_new_i', 'bq1_new_j',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8 추가 0.491**
## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8,12,13,14 추가 0.503**

* bq13,14은 각각 보면 버리는게 맞음 근데 둘을 뺐을때 0.484

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

In [ ]:
importances = clfs[3].feature_importances_
forest_importances = pd.Series(importances, index=X20.columns)
plt.rcParams["figure.figsize"] = (20,10)
forest_importances.plot.bar()

fig.tight_layout()

## 20 bq1 범주 빼고  실수 넣고 0.461

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       'bq1_new_a', 'bq1_new_b',
       'bq1_new_c', 'bq1_new_d', 'bq1_new_e', 'bq1_new_f', 'bq1_new_g',
       'bq1_new_h', 'bq1_new_i', 'bq1_new_j' ],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       'bq1_new_a', 'bq1_new_b',
       'bq1_new_c', 'bq1_new_d', 'bq1_new_e', 'bq1_new_f', 'bq1_new_g',
       'bq1_new_h', 'bq1_new_i', 'bq1_new_j'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## 20 학과 빼기 0.4539

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       '학과_기타', '학과_문과', '학과_예체능',
       '학과_의료', '학과_이과'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       '학과_기타', '학과_문과', '학과_예체능',
       '학과_의료', '학과_이과'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## 20 saq 전부 원래대로(범주 제거) 0.474

In [ ]:
tr20_del1=tr20_all.drop(['bq1',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       'saq1',
       'saq2', 'saq3', 'saq4', 'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10',
       'saq11', 'saq12', 'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18',
       'saq19', 'saq20', 'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26',
       'saq27', 'saq28', 'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34',
       'saq35', 'saq36', 'saq37', 'saq38', 'saq39', 'saq40', 'saq41', 'saq42',
       'saq43', 'saq44'],axis=1)
ts20_del1=ts20_all.drop(['bq1',
       'bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',
       'saq1',
       'saq2', 'saq3', 'saq4', 'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10',
       'saq11', 'saq12', 'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18',
       'saq19', 'saq20', 'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26',
       'saq27', 'saq28', 'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34',
       'saq35', 'saq36', 'saq37', 'saq38', 'saq39', 'saq40', 'saq41', 'saq42',
       'saq43', 'saq44'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

## 20 bq8~ 14 원래대로 0.458

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1','bq8_sum',
       'bq12_sum', 'bq13_sum', 'bq14_sum',],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1','bq8_sum',
       'bq12_sum', 'bq13_sum', 'bq14_sum',],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)


## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8,12,13,14 추가 

## xgboost


In [ ]:
df17=pd.read_csv('/content/drive/MyDrive/잡데이터제출/pred17.csv')
df18=pd.read_csv('/content/drive/MyDrive/잡데이터제출/pred18.csv')

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1'],axis=1)

In [ ]:
df=pd.get_dummies(tr20_del1[['bq2','bq27_2']])
tr20_del1=pd.concat([tr20_del1,df],axis=1)
tr20_del1=tr20_del1.drop(['bq2','bq27_2'],axis=1)


In [ ]:
from xgboost import XGBClassifier # model
y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance
# dtrain = xgb.DMatrix(data=X_train, label = y_train)
# dtest = xgb.DMatrix(data=X_test, label=y_test)
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f1)

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (14, 20)
plot_importance(model)
plt.show()

## xgboost cv


In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1'],axis=1)

In [ ]:
df=pd.get_dummies(tr20_del1[['bq2','bq27_2']])
tr20_del1=pd.concat([tr20_del1,df],axis=1)
tr20_del1=tr20_del1.drop(['bq2','bq27_2'],axis=1)


In [ ]:
from xgboost import XGBClassifier # model
y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance
# dtrain = xgb.DMatrix(data=X_train, label = y_train)
# dtest = xgb.DMatrix(data=X_test, label=y_test)
xgb = XGBClassifier()
scores = cross_val_score(xgb, X_train, y_train.ravel(), cv=5, scoring = 'f1')
clf = xgb.fit(X_train, y_train.ravel())
y_pred = clf.predict(X_test)
f1 = f1_score(Y_test, y_pred, average='macro')
print (f1)

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (14, 20)
plot_importance(xgb)
plt.show()

## xgboost cv


In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1','bq8_1', 'bq8_2', 'bq8_3',
       'bq12_1', 'bq12_2', 'bq12_3', 'bq12_4', 'bq12_5',
       'bq13_1', 'bq13_2', 'bq13_3', 
       'bq14_1','bq14_2', 'bq14_3', 'bq14_4', 'bq14_5', 'bq14_6', 'bq14_7',],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2','bq1'],axis=1)

In [ ]:
df=pd.get_dummies(tr20_del1[['bq2','bq27_2']])
tr20_del1=pd.concat([tr20_del1,df],axis=1)
tr20_del1=tr20_del1.drop(['bq2','bq27_2'],axis=1)


In [ ]:
from xgboost import XGBClassifier # model
y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance
# dtrain = xgb.DMatrix(data=X_train, label = y_train)
# dtest = xgb.DMatrix(data=X_test, label=y_test)
xgb = XGBClassifier()
scores = cross_val_score(xgb, X_train, y_train.ravel(), cv=5, scoring = 'f1')
clf = xgb.fit(X_train, y_train.ravel())
y_pred = clf.predict(X_test)
f1 = f1_score(Y_test, y_pred, average='macro')
print (f1)

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (14, 20)
plot_importance(xgb)
plt.show()

## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8 추가 0.491**
## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8,12,13,14 추가 0.503**

* bq13,14은 각각 보면 버리는게 맞음 근데 둘을 뺐을때 0.484

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2'],axis=1)

In [ ]:

y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
from keras import backend as K
from sklearn.metrics import recall_score 

from sklearn.metrics import precision_score 

from sklearn.metrics import f1_score 
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [ ]:
len(y20.unique())

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

model = tf.keras.Sequential([ 
                             tf.keras.layers.Dense(units=1000, activation='relu', input_shape=(152,)), 
                             tf.keras.layers.Dense(units=800, activation='relu'), 
                             tf.keras.layers.Dense(units=600, activation='relu'), 
                             tf.keras.layers.Dense(units=537, activation='relu') ]) 
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='categorical_crossentropy',
              metrics=['accuracy',precision, recall, f1score]) 
model.summary()


In [ ]:
history = model.fit(X_train, Y_train, validation_split=0.3, epochs=200, batch_size=5, verbose=1) 

In [ ]:
_loss, _acc, _precision, _recall, _f1score = model.evaluate(X_test, Y_test, batch_size=30, verbose=0)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = model.fit(X_train, Y_train, validation_split=0.3, epochs=10, verbose=0)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

In [ ]:
importances = clfs[3].feature_importances_
forest_importances = pd.Series(importances, index=X20.columns)
plt.rcParams["figure.figsize"] = (20,10)
forest_importances.plot.bar()

fig.tight_layout()

## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8 추가 0.491**
## **20 bq1알파벳(a~j) +원래범주(obj21) + bq8,12,13,14 추가 0.503**

* bq13,14은 각각 보면 버리는게 맞음 근데 둘을 뺐을때 0.484

In [ ]:
tr20_del1=tr20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2'],axis=1)
ts20_del1=ts20_all.drop(['saq1_1', 'saq1_2', 'saq2_1', 'saq2_2', 'saq3_1', 'saq3_2', 'saq4_1',
       'saq4_2', 'saq5_1', 'saq5_2', 'saq6_1', 'saq6_2', 'saq7_1', 'saq7_2',
       'saq8_1', 'saq8_2', 'saq9_1', 'saq9_2', 'saq10_1', 'saq10_2', 'saq11_1',
       'saq11_2', 'saq12_1', 'saq12_2', 'saq13_1', 'saq13_2', 'saq14_1',
       'saq14_2', 'saq15_1', 'saq15_2', 'saq16_1', 'saq16_2', 'saq17_1',
       'saq17_2', 'saq18_1', 'saq18_2', 'saq19_1', 'saq19_2', 'saq20_1',
       'saq20_2', 'saq21_1', 'saq21_2', 'saq22_1', 'saq22_2', 'saq23_1',
       'saq23_2', 'saq24_1', 'saq24_2', 'saq25_1', 'saq25_2', 'saq26_1',
       'saq26_2', 'saq27_1', 'saq27_2', 'saq28_1', 'saq28_2', 'saq29_1',
       'saq29_2', 'saq30_1', 'saq30_2', 'saq31_1', 'saq31_2', 'saq32_1',
       'saq32_2', 'saq33_1', 'saq33_2', 'saq34_1', 'saq34_2', 'saq35_1',
       'saq35_2', 'saq36_1', 'saq36_2', 'saq37_1', 'saq37_2', 'saq38_1',
       'saq38_2', 'saq39_1', 'saq39_2', 'saq40_1', 'saq40_2', 'saq41_1',
       'saq41_2', 'saq42_1', 'saq42_2', 'saq43_1', 'saq43_2', 'saq44_1',
       'saq44_2'],axis=1)

In [ ]:
tr20_del1.columns[50:150]

In [ ]:
cor=tr20_del1[['vq1', 'vq2', 'vq3', 'vq4', 'vq5', 'vq6', 'vq7', 'vq8', 'vq9', 'vq10',
       'vq11', 'vq12', 'vq13']].corr()
cor

In [ ]:
tr20_all[['bq25', 'bq29', 'bq30_1',
       'bq30_2', 'bq30_3']].head()

In [ ]:
# tr20_all[['saq1', 'saq2', 'saq3', 'saq4',
#        'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10', 'saq11', 'saq12',
#        'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18', 'saq19', 'saq20',
#        'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26']].corr()
# tr20_all[['saq27', 'saq28',
#        'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34', 'saq35', 'saq36',
#        'saq37', 'saq38']].corr()
tr20_all[['saq39', 'saq40', 'saq41', 'saq42']].corr()
# 'saq43', 'saq44'
#tr20_all[['saq1', 'saq2', 'saq3', 'saq4',
      #  'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10', 'saq11', 'saq12',
      #  'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18', 'saq19', 'saq20',
      #  'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26', 'saq27', 'saq28',
      #  'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34', 'saq35', 'saq36',
      #  'saq37', 'saq38', 'saq39', 'saq40', 'saq41', 'saq42', 'saq43', 'saq44']].corr()

In [ ]:
tr20_del1[['bq30_1', 'bq30_2', 'bq30_3']].describe()

In [ ]:
tr20_all['bq30_1_cut'] = 0
tr20_all.loc[tr20_all['bq30_1'] >= 2000, 'bq30_1_cut'] = 1
tr20_all.loc[tr20_all['bq30_1'] >= 3000, 'bq30_1_cut'] = 2
tr20_all.loc[tr20_all['bq30_1'] >= 4000, 'bq30_1_cut'] = 3
tr20_all.loc[tr20_all['bq30_1'] >= 5000, 'bq30_1_cut'] = 4
tr20_all.loc[tr20_all['bq30_1'] >= 6000, 'bq30_1_cut'] = 5
tr20_all.loc[tr20_all['bq30_1'] >= 7000, 'bq30_1_cut'] = 6
tr20_all.loc[tr20_all['bq30_1'] >= 8000, 'bq30_1_cut'] = 7
tr20_all.loc[tr20_all['bq30_1'] >= 9000, 'bq30_1_cut'] = 8
tr20_all.loc[tr20_all['bq30_1'] >= 10000, 'bq30_1_cut'] = 9
tr20_all['bq30_2_cut'] = 0
tr20_all.loc[tr20_all['bq30_2'] >= 2000, 'bq30_2_cut'] = 1
tr20_all.loc[tr20_all['bq30_2'] >= 3000, 'bq30_2_cut'] = 2
tr20_all.loc[tr20_all['bq30_2'] >= 4000, 'bq30_2_cut'] = 3
tr20_all.loc[tr20_all['bq30_2'] >= 5000, 'bq30_2_cut'] = 4
tr20_all.loc[tr20_all['bq30_2'] >= 6000, 'bq30_2_cut'] = 5
tr20_all.loc[tr20_all['bq30_2'] >= 7000, 'bq30_2_cut'] = 6
tr20_all.loc[tr20_all['bq30_2'] >= 8000, 'bq30_2_cut'] = 7
tr20_all.loc[tr20_all['bq30_2'] >= 9000, 'bq30_2_cut'] = 8
tr20_all.loc[tr20_all['bq30_2'] >= 10000, 'bq30_2_cut'] = 9
tr20_all['bq30_3_cut'] = 0
tr20_all.loc[tr20_all['bq30_3'] >= 2000, 'bq30_3_cut'] = 1
tr20_all.loc[tr20_all['bq30_3'] >= 3000, 'bq30_3_cut'] = 2
tr20_all.loc[tr20_all['bq30_3'] >= 4000, 'bq30_3_cut'] = 3
tr20_all.loc[tr20_all['bq30_3'] >= 5000, 'bq30_3_cut'] = 4
tr20_all.loc[tr20_all['bq30_3'] >= 6000, 'bq30_3_cut'] = 5
tr20_all.loc[tr20_all['bq30_3'] >= 7000, 'bq30_3_cut'] = 6
tr20_all.loc[tr20_all['bq30_3'] >= 8000, 'bq30_3_cut'] = 7
tr20_all.loc[tr20_all['bq30_3'] >= 9000, 'bq30_3_cut'] = 8
tr20_all.loc[tr20_all['bq30_3'] >= 10000, 'bq30_3_cut'] = 9

ts20_all['bq30_1_cut'] = 0
ts20_all.loc[ts20_all['bq30_1'] >= 2000, 'bq30_1_cut'] = 1
ts20_all.loc[ts20_all['bq30_1'] >= 3000, 'bq30_1_cut'] = 2
ts20_all.loc[ts20_all['bq30_1'] >= 4000, 'bq30_1_cut'] = 3
ts20_all.loc[ts20_all['bq30_1'] >= 5000, 'bq30_1_cut'] = 4
ts20_all.loc[ts20_all['bq30_1'] >= 6000, 'bq30_1_cut'] = 5
ts20_all.loc[ts20_all['bq30_1'] >= 7000, 'bq30_1_cut'] = 6
ts20_all.loc[ts20_all['bq30_1'] >= 8000, 'bq30_1_cut'] = 7
ts20_all.loc[ts20_all['bq30_1'] >= 9000, 'bq30_1_cut'] = 8
ts20_all.loc[ts20_all['bq30_1'] >= 10000, 'bq30_1_cut'] = 9
ts20_all['bq30_2_cut'] = 0
ts20_all.loc[ts20_all['bq30_2'] >= 2000, 'bq30_2_cut'] = 1
ts20_all.loc[ts20_all['bq30_2'] >= 3000, 'bq30_2_cut'] = 2
ts20_all.loc[ts20_all['bq30_2'] >= 4000, 'bq30_2_cut'] = 3
ts20_all.loc[ts20_all['bq30_2'] >= 5000, 'bq30_2_cut'] = 4
ts20_all.loc[ts20_all['bq30_2'] >= 6000, 'bq30_2_cut'] = 5
ts20_all.loc[ts20_all['bq30_2'] >= 7000, 'bq30_2_cut'] = 6
ts20_all.loc[ts20_all['bq30_2'] >= 8000, 'bq30_2_cut'] = 7
ts20_all.loc[ts20_all['bq30_2'] >= 9000, 'bq30_2_cut'] = 8
ts20_all.loc[ts20_all['bq30_2'] >= 10000, 'bq30_2_cut'] = 9
ts20_all['bq30_3_cut'] = 0
ts20_all.loc[ts20_all['bq30_3'] >= 2000, 'bq30_3_cut'] = 1
ts20_all.loc[ts20_all['bq30_3'] >= 3000, 'bq30_3_cut'] = 2
ts20_all.loc[ts20_all['bq30_3'] >= 4000, 'bq30_3_cut'] = 3
ts20_all.loc[ts20_all['bq30_3'] >= 5000, 'bq30_3_cut'] = 4
ts20_all.loc[ts20_all['bq30_3'] >= 6000, 'bq30_3_cut'] = 5
ts20_all.loc[ts20_all['bq30_3'] >= 7000, 'bq30_3_cut'] = 6
ts20_all.loc[ts20_all['bq30_3'] >= 8000, 'bq30_3_cut'] = 7
ts20_all.loc[ts20_all['bq30_3'] >= 9000, 'bq30_3_cut'] = 8
ts20_all.loc[ts20_all['bq30_3'] >= 10000, 'bq30_3_cut'] = 9

In [ ]:
tr20_all['bq25_cut'] = 0
tr20_all.loc[tr20_all['bq25'] >= 20, 'bq25_cut'] = 1
tr20_all.loc[tr20_all['bq25'] >= 30, 'bq25_cut'] = 2
tr20_all.loc[tr20_all['bq25'] >= 40, 'bq25_cut'] = 3
tr20_all.loc[tr20_all['bq25'] >= 50, 'bq25_cut'] = 4
tr20_all.loc[tr20_all['bq25'] >= 60, 'bq25_cut'] = 5
tr20_all.loc[tr20_all['bq25'] >= 70, 'bq25_cut'] = 6
tr20_all.loc[tr20_all['bq25'] >= 80, 'bq25_cut'] = 7
tr20_all.loc[tr20_all['bq25'] >= 90, 'bq25_cut'] = 8

tr20_all['bq29_cut'] = 0
tr20_all.loc[tr20_all['bq29'] >= 20, 'bq29_cut'] = 1
tr20_all.loc[tr20_all['bq29'] >= 30, 'bq29_cut'] = 2
tr20_all.loc[tr20_all['bq29'] >= 40, 'bq29_cut'] = 3
tr20_all.loc[tr20_all['bq29'] >= 50, 'bq29_cut'] = 4
tr20_all.loc[tr20_all['bq29'] >= 60, 'bq29_cut'] = 5
tr20_all.loc[tr20_all['bq29'] >= 70, 'bq29_cut'] = 6
tr20_all.loc[tr20_all['bq29'] >= 80, 'bq29_cut'] = 7
tr20_all.loc[tr20_all['bq29'] >= 90, 'bq29_cut'] = 8

ts20_all['bq25_cut'] = 0
ts20_all.loc[ts20_all['bq25'] >= 20, 'bq25_cut'] = 1
ts20_all.loc[ts20_all['bq25'] >= 30, 'bq25_cut'] = 2
ts20_all.loc[ts20_all['bq25'] >= 40, 'bq25_cut'] = 3
ts20_all.loc[ts20_all['bq25'] >= 50, 'bq25_cut'] = 4
ts20_all.loc[ts20_all['bq25'] >= 60, 'bq25_cut'] = 5
ts20_all.loc[ts20_all['bq25'] >= 70, 'bq25_cut'] = 6
ts20_all.loc[ts20_all['bq25'] >= 80, 'bq25_cut'] = 7
ts20_all.loc[ts20_all['bq25'] >= 90, 'bq25_cut'] = 8

ts20_all['bq29_cut'] = 0
ts20_all.loc[ts20_all['bq29'] >= 20, 'bq29_cut'] = 1
ts20_all.loc[ts20_all['bq29'] >= 30, 'bq29_cut'] = 2
ts20_all.loc[ts20_all['bq29'] >= 40, 'bq29_cut'] = 3
ts20_all.loc[ts20_all['bq29'] >= 50, 'bq29_cut'] = 4
ts20_all.loc[ts20_all['bq29'] >= 60, 'bq29_cut'] = 5
ts20_all.loc[ts20_all['bq29'] >= 70, 'bq29_cut'] = 6
ts20_all.loc[ts20_all['bq29'] >= 80, 'bq29_cut'] = 7
ts20_all.loc[ts20_all['bq29'] >= 90, 'bq29_cut'] = 8

In [ ]:
ts20_all=ts20_all.drop(['bq1_new_a', 'bq1_new_b', 'bq1_new_c',
       'bq1_new_d', 'bq1_new_e', 'bq1_new_f', 'bq1_new_g', 'bq1_new_h',
       'bq1_new_i', 'bq1_new_j', '학과_기타', '학과_문과', '학과_예체능', '학과_의료', '학과_이과'],axis=1)
tr20_all=tr20_all.drop(['bq1_new_a', 'bq1_new_b', 'bq1_new_c',
       'bq1_new_d', 'bq1_new_e', 'bq1_new_f', 'bq1_new_g', 'bq1_new_h',
       'bq1_new_i', 'bq1_new_j', '학과_기타', '학과_문과', '학과_예체능', '학과_의료', '학과_이과'],axis=1)

In [ ]:
tr20_all.columns[200:]

In [ ]:
ts20_all.columns[200:]

In [ ]:
tr20_all.to_csv('/content/drive/MyDrive/조달청 공모전/tr20_all.csv', index=False)
ts20_all.to_csv('/content/drive/MyDrive/조달청 공모전/ts20_all.csv', index=False)

In [ ]:
# 그나마 vq 아래로 대체하는게 나음
df1=tr20_del1[['vq1', 'vq2', 'vq3', 'vq4', 'vq5', 'vq6', 'vq7', 'vq8', 'vq9', 'vq10',
       'vq11', 'vq12', 'vq13']]
df2=tr20_del1[['saq1', 'saq2', 'saq3', 'saq4',
       'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10', 'saq11', 'saq12',
       'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18', 'saq19', 'saq20',
       'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26']] 
df3=tr20_del1[['saq27', 'saq28',
       'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34', 'saq35', 'saq36',
       'saq37', 'saq38']]
df4=tr20_del1[['saq39', 'saq40', 'saq41', 'saq42']]
# df5=tr20_del1[['saq43', 'saq44']]
# tr20_del1["vq_mean"] = df1.mean(axis = 1) 
# tr20_del1["vq_var"] = df1.var(axis = 1) 
# tr20_del1["vq_std"] = df1.std(axis = 1) 
# tr20_del1["vq_sum"] = df1.sum(axis = 1) 
#tr20_del1["vq_median"] = df1.median(axis = 1)
tr20_del1["saq1_mean"] = df2.mean(axis = 1) 
tr20_del1["saq1_var"] = df2.var(axis = 1) 
# tr20_del1["saq1_std"] = df2.std(axis = 1) 
# tr20_del1["saq1_sum"] = df2.sum(axis = 1) 
#tr20_del1["saq1_median"] = df2.median(axis = 1)
tr20_del1["saq2_mean"] = df3.mean(axis = 1) 
tr20_del1["saq2_var"] = df3.var(axis = 1) 
# tr20_del1["saq2_std"] = df3.std(axis = 1) 
# tr20_del1["saq2_sum"] = df3.sum(axis = 1) 
tr20_del1["saq2_median"] = df3.median(axis = 1)
tr20_del1["saq3_mean"] = df4.mean(axis = 1) 
tr20_del1["saq3_var"] = df4.var(axis = 1) 
# tr20_del1["saq3_std"] = df4.std(axis = 1) 
# tr20_del1["saq3_sum"] = df4.sum(axis = 1) 
#tr20_del1["saq3_median"] = df4.median(axis = 1)


tr20_del1=tr20_del1.drop(['saq1', 'saq2',
       'saq3', 'saq4', 'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10',
       'saq11', 'saq12', 'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18',
       'saq19', 'saq20', 'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26',
       'saq27', 'saq28', 'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34',
       'saq35', 'saq36', 'saq37', 'saq38', 'saq39', 'saq40', 'saq41', 'saq42'],axis=1)

In [ ]:
df1=ts20_del1[['vq1', 'vq2', 'vq3', 'vq4', 'vq5', 'vq6', 'vq7', 'vq8', 'vq9', 'vq10',
       'vq11', 'vq12', 'vq13']]
df2=ts20_del1[['saq1', 'saq2', 'saq3', 'saq4',
       'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10', 'saq11', 'saq12',
       'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18', 'saq19', 'saq20',
       'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26']] 
df3=ts20_del1[['saq27', 'saq28',
       'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34', 'saq35', 'saq36',
       'saq37', 'saq38']]
df4=ts20_del1[['saq39', 'saq40', 'saq41', 'saq42']]
# df5=ts20_del1[['saq43', 'saq44']]
ts20_del1["vq_mean"] = df1.mean(axis = 1) 
ts20_del1["vq_var"] = df1.var(axis = 1) 
#ts20_del1["vq_std"] = df1.std(axis = 1) 
#ts20_del1["vq_sum"] = df1.sum(axis = 1) 
#ts20_del1["vq_median"] = df1.median(axis = 1)
ts20_del1["saq1_mean"] = df2.mean(axis = 1) 
ts20_del1["saq1_var"] = df2.var(axis = 1) 
#ts20_del1["saq1_std"] = df2.std(axis = 1) 
#ts20_del1["saq1_sum"] = df2.sum(axis = 1) 
#ts20_del1["saq1_median"] = df2.median(axis = 1)
ts20_del1["saq2_mean"] = df3.mean(axis = 1) 
ts20_del1["saq2_var"] = df3.var(axis = 1) 
#ts20_del1["saq2_std"] = df3.std(axis = 1) 
#ts20_del1["saq2_sum"] = df3.sum(axis = 1) 
#ts20_del1["saq2_median"] = df3.median(axis = 1)
ts20_del1["saq3_mean"] = df4.mean(axis = 1) 
ts20_del1["saq3_var"] = df4.var(axis = 1) 
#ts20_del1["saq3_std"] = df4.std(axis = 1) 
#ts20_del1["saq3_sum"] = df4.sum(axis = 1) 
#ts20_del1["saq3_median"] = df4.median(axis = 1)


ts20_del1=ts20_del1.drop(['vq1', 'vq2', 'vq3', 'vq4', 'vq5', 'vq6', 'vq7', 'vq8', 'vq9', 'vq10',
       'vq11', 'vq12', 'vq13','saq1', 'saq2',
       'saq3', 'saq4', 'saq5', 'saq6', 'saq7', 'saq8', 'saq9', 'saq10',
       'saq11', 'saq12', 'saq13', 'saq14', 'saq15', 'saq16', 'saq17', 'saq18',
       'saq19', 'saq20', 'saq21', 'saq22', 'saq23', 'saq24', 'saq25', 'saq26',
       'saq27', 'saq28', 'saq29', 'saq30', 'saq31', 'saq32', 'saq33', 'saq34',
       'saq35', 'saq36', 'saq37', 'saq38', 'saq39', 'saq40', 'saq41', 'saq42'],axis=1)

In [ ]:
X20[['bq1']].describe()

In [ ]:
y20=tr20_all['knowcode']; X20=tr20_del1.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(random_state=3)
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1) # 466

In [ ]:
importances = clfs[3].feature_importances_
forest_importances = pd.Series(importances, index=X20.columns)
plt.rcParams["figure.figsize"] = (20,10)
forest_importances.plot.bar()

fig.tight_layout()

## 17 all 모델링

In [ ]:
y17=tr17_all['knowcode']; X17=tr17_all.drop('knowcode', axis=1, inplace=False)
y18=tr18_all['knowcode']; X18=tr18_all.drop('knowcode', axis=1, inplace=False)
y19=tr19_all['knowcode']; X19=tr19_all.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_all.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20, random_state=3)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

##  17 del 모델링

In [ ]:
y17=tr17_del['knowcode']; X17=tr17_del.drop('knowcode', axis=1, inplace=False)
y18=tr18_del['knowcode']; X18=tr18_del.drop('knowcode', axis=1, inplace=False)
y19=tr19_del['knowcode']; X19=tr19_del.drop('knowcode', axis=1, inplace=False)
y20=tr20_del['knowcode']; X20=tr20_del.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X17, y17, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

# 모델링

## 18 all 모델링

In [ ]:
y17=tr17_all['knowcode']; X17=tr17_all.drop('knowcode', axis=1, inplace=False)
y18=tr18_all['knowcode']; X18=tr18_all.drop('knowcode', axis=1, inplace=False)
y19=tr19_all['knowcode']; X19=tr19_all.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_all.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X18, y18, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

##  18 del 모델링

In [ ]:
y17=tr17_del['knowcode']; X17=tr17_del.drop('knowcode', axis=1, inplace=False)
y18=tr18_del['knowcode']; X18=tr18_del.drop('knowcode', axis=1, inplace=False)
y19=tr19_del['knowcode']; X19=tr19_del.drop('knowcode', axis=1, inplace=False)
y20=tr20_del['knowcode']; X20=tr20_del.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X18, y18, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

# 모델링

## 19 all 모델링

In [ ]:
y17=tr17_all['knowcode']; X17=tr17_all.drop('knowcode', axis=1, inplace=False)
y18=tr18_all['knowcode']; X18=tr18_all.drop('knowcode', axis=1, inplace=False)
y19=tr19_all['knowcode']; X19=tr19_all.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_all.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X19, y19, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

##  19 del 모델링

In [ ]:
y17=tr17_del['knowcode']; X17=tr17_del.drop('knowcode', axis=1, inplace=False)
y18=tr18_del['knowcode']; X18=tr18_del.drop('knowcode', axis=1, inplace=False)
y19=tr19_del['knowcode']; X19=tr19_del.drop('knowcode', axis=1, inplace=False)
y20=tr20_del['knowcode']; X20=tr20_del.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X19, y19, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

# 모델링

## 20 all 모델링

In [ ]:
y17=tr17_all['knowcode']; X17=tr17_all.drop('knowcode', axis=1, inplace=False)
y18=tr18_all['knowcode']; X18=tr18_all.drop('knowcode', axis=1, inplace=False)
y19=tr19_all['knowcode']; X19=tr19_all.drop('knowcode', axis=1, inplace=False)
y20=tr20_all['knowcode']; X20=tr20_all.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)

##  20 del 모델링

In [ ]:
y17=tr17_del['knowcode']; X17=tr17_del.drop('knowcode', axis=1, inplace=False)
y18=tr18_del['knowcode']; X18=tr18_del.drop('knowcode', axis=1, inplace=False)
y19=tr19_del['knowcode']; X19=tr19_del.drop('knowcode', axis=1, inplace=False)
y20=tr20_del['knowcode']; X20=tr20_del.drop('knowcode', axis=1, inplace=False)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X20, y20, test_size = 0.20)

In [ ]:
clfs = [KNeighborsClassifier(n_neighbors = 1), # 버려
      GaussianNB(),                            # 버려
      DecisionTreeClassifier(splitter = 'random'), # 버려
       RandomForestClassifier(),
        GradientBoostingClassifier(),
        LGBMClassifier()
        ]
clfName = {1:'KNN', 2:'Naive Bayes', 3:'Decision Trees', 4:'Random Forests', 5:'GradientBoostingClassifier',6:'LGBMClassifier'}

for i in range(len(clfs)):
    scores = cross_val_score(clfs[i], X_train, Y_train.ravel(), cv=5, scoring = 'f1')
    print ("mean validation F1 for",clfName[i+1])
    clf = clfs[i].fit(X_train, Y_train.ravel())
    Y_pred = clf.predict(X_test)
    f1 = f1_score(Y_test, Y_pred, average='macro')
    print ('Test F1 for',clfName[i+1], ":", f1)